## Claude Sonnet 4
Claude Sonnet 4 är Anthropics mest avancerade modell med förbättrad kontextförståelse, resonemang och högre kvalitet i strukturerad extraktion jämfört med tidigare versioner. Vi kommer att utvärdera dessa förbättringar i vår testning för att se hur modellen presterar på våra PDF-analyser.

In [1]:
!pip install anthropic pdfplumber rapidfuzz jiwer psutil


In [ ]:
import os
import time
import psutil
import pdfplumber
from anthropic import Anthropic
from rapidfuzz import fuzz
from jiwer import wer, cer
import re
import json

os.environ["ANTHROPIC_API_KEY"] = input("Enter Anthropic API Key: ")

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])


### Ladda upp PDF och gold.txt i Colab

In [4]:
from google.colab import files
print("Ladda upp PDF som ska benchmarkas:")
uploaded_pdf = files.upload()

pdf_path = list(uploaded_pdf.keys())[0]
print("PDF:", pdf_path)


Ladda upp PDF som ska benchmarkas:


Saving pdf_test.pdf to pdf_test.pdf
PDF: pdf_test.pdf


In [5]:

print("Ladda upp GOLD.txt (Docling-export)")
uploaded_gt = files.upload()

gold_path = list(uploaded_gt.keys())[0]

with open(gold_path, "r") as f:
    gold_md = f.read()

print("GOLD loaded:", gold_path)

Ladda upp GOLD.txt (Docling-export)


Saving gold.txt to gold.txt
GOLD loaded: gold.txt


### Läs PDF till full text + chunking

In [12]:
def extract_pdf_text(pdf_path):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for p in pdf.pages:
            pages.append(p.extract_text() or "")
    return "\n\n".join(pages)

def chunk_text(text, max_chars=100000):
    """
    max_chars=100000 fungerar bra för Opus och undviker rate limits.
    Justera vid behov.
    """
    return [text[i:i+max_chars] for i in range(0, len(text), max_chars)]


### Kör Opus på hela PDF:en (chunkad)

In [13]:
def run_opus_on_pdf(full_text, out_path="opus_output.md", max_chars=15000):
    chunks = chunk_text(full_text, max_chars=max_chars)
    outputs = []

    print(f" Modell: Claude Sonnet 4")
    print(f" Antal chunkar: {len(chunks)}")

    proc = psutil.Process()
    start_mem = proc.memory_info().rss
    start_time = time.time()

    for i, ch in enumerate(chunks):
        print(f"  → Kör chunk {i+1}/{len(chunks)}")

        prompt = f"""
Extrahera struktur säkert och returnera ENDAST ren Markdown.
Detta är chunk {i+1} av {len(chunks)} från PDF-texten.

Text:
{ch}
"""

        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=8000,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )

        outputs.append(response.content[0].text)

    end_time = time.time()
    end_mem = proc.memory_info().rss

    full_md = "\n\n".join(outputs)

    with open(out_path, "w") as f:
        f.write(full_md)

    metrics = {
        "time_s": end_time - start_time,
        "memory_MB": (end_mem - start_mem) / (1024*1024),
        "num_chunks": len(chunks),
    }

    return full_md, metrics


### Utvärdering
- WER / CER
- Similarity
- Strukturindikator

In [14]:
def evaluate_against_gold(pred, gold_path="gold.txt"):
    with open(gold_path, "r") as f:
        gold = f.read()

    return {
        "similarity": float(fuzz.token_set_ratio(pred, gold)),
        "WER": float(wer(gold, pred)),
        "CER": float(cer(gold, pred))
    }


def structure_score(md):
    score = 0
    score += 1 if re.search(r"^# ", md, re.M) else 0
    score += 1 if re.search(r"^## ", md, re.M) else 0
    score += 1 if re.search(r"(^- |^\* |^\d+\. )", md, re.M) else 0  # listor
    score += 1 if "|" in md else 0
    score += 1 if "```" in md else 0                      # kodblock
    return score


### Kör Opus på hela pdf:en + beräkna alla metrikvärden

In [15]:
pdf_text = extract_pdf_text("pdf_test.pdf")

opus_md, opus_perf = run_opus_on_pdf(pdf_text)
opus_eval = evaluate_against_gold(opus_md, gold_path="gold.txt")
opus_struct = structure_score(opus_md)

result = {
    "opus": {
        "performance": opus_perf,
        "evaluation": opus_eval,
        "structure_score": opus_struct
    }
}

print(json.dumps(result, indent=2, ensure_ascii=False))


 Modell: Claude Sonnet 4
 Antal chunkar: 14
  → Kör chunk 1/14
  → Kör chunk 2/14
  → Kör chunk 3/14
  → Kör chunk 4/14
  → Kör chunk 5/14
  → Kör chunk 6/14
  → Kör chunk 7/14
  → Kör chunk 8/14
  → Kör chunk 9/14
  → Kör chunk 10/14
  → Kör chunk 11/14
  → Kör chunk 12/14
  → Kör chunk 13/14
  → Kör chunk 14/14
{
  "opus": {
    "performance": {
      "time_s": 1176.3357138633728,
      "memory_MB": 0.453125,
      "num_chunks": 14
    },
    "evaluation": {
      "similarity": 90.41038492220171,
      "WER": 0.4697742170429716,
      "CER": 0.4966562785383453
    },
    "structure_score": 4
  }
}


In [28]:
#Tabell med resultat
import pandas as pd

opus_row = {
    "time_s": opus_perf["time_s"],
    "memory_MB": opus_perf["memory_MB"],
    "num_chunks": opus_perf["num_chunks"],
    "similarity": opus_eval["similarity"],
    "WER": opus_eval["WER"],
    "CER": opus_eval["CER"],
    "structure_score": opus_struct
}

df = pd.DataFrame([opus_row], index=["Opus"])
df


,time_s,memory_MB,num_chunks,similarity,WER,CER,structure_score
Opus,1176.335714,0.453125,14,90.410385,0.469774,0.496656,4


### EXTRA EVALUATION

In [32]:
!pip install --upgrade rapidfuzz


In [37]:
from rapidfuzz.distance.LCSseq import normalized_similarity


In [39]:
import re
import numpy as np
from rapidfuzz.distance import Levenshtein

# Token ratio (över/under-extraktion)
def token_ratio(pred, gold):
    pred_tokens = pred.split()
    gold_tokens = gold.split()
    return len(pred_tokens) / max(len(gold_tokens), 1)

# Rubrikstruktur-precision
def heading_stats(md):
    h1 = len(re.findall(r'^# ', md, flags=re.M))
    h2 = len(re.findall(r'^## ', md, flags=re.M))
    h3 = len(re.findall(r'^### ', md, flags=re.M))
    return {"H1": h1, "H2": h2, "H3": h3}

# ROUGE-L approximation (longest common subsequence ratio)
def rouge_l(pred, gold):
    from rapidfuzz.distance.LCSseq import normalized_similarity
    return normalized_similarity(pred, gold)

# Semantisk similarity med cosine på token-frekvenser (lightweight)
def semantic_similarity(pred, gold):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vect = TfidfVectorizer().fit([pred, gold])
    X = vect.transform([pred, gold]).toarray()
    denom = (np.linalg.norm(X[0]) * np.linalg.norm(X[1]))
    return float(np.dot(X[0], X[1]) / denom) if denom != 0 else 0.0

# Hallucination rate (ord som inte finns i PDF alls)
def hallucination_rate(pred, gold):
    gold_words = set(gold.lower().split())
    pred_words = pred.lower().split()
    hallu = [w for w in pred_words if w not in gold_words]
    return len(hallu) / max(len(pred_words), 1)

#Layout preservation score (baserat på blocktyper)
def layout_score(md):
    bullets = len(re.findall(r'^[-*] ', md, flags=re.M))
    numbers = len(re.findall(r'^\d+\. ', md, flags=re.M))
    tables = md.count("|")
    codeblocks = md.count("```")
    return {
        "bullet_points": bullets,
        "numbered_lists": numbers,
        "tables": tables,
        "codeblocks": codeblocks
    }


In [40]:
opus_eval = evaluate_against_gold(gold_md)
opus_struct = structure_score(opus_md)


In [43]:
with open("gold.txt", "r") as f:
    gold = f.read()

pred = opus_md

extra = {
    "token_ratio": token_ratio(pred, gold),
    "heading_stats": heading_stats(pred),
    "rouge_l": rouge_l(pred, gold),
    "semantic_similarity": semantic_similarity(pred, gold),
    "hallucination_rate": hallucination_rate(pred, gold),
    "layout_score": layout_score(pred)
}

print("\n EXTRA EVALUATION RESULTS:")
import json
print(json.dumps(extra, indent=2, ensure_ascii=False))



 EXTRA EVALUATION RESULTS:
{
  "token_ratio": 0.8621952460072565,
  "heading_stats": {
    "H1": 39,
    "H2": 130,
    "H3": 145
  },
  "rouge_l": 0.582868188165724,
  "semantic_similarity": 0.9884855127226093,
  "hallucination_rate": 0.06091949502497345,
  "layout_score": {
    "bullet_points": 223,
    "numbered_lists": 11,
    "tables": 2912,
    "codeblocks": 0
  }
}


### Läsa och printa .md som vanlig text:

In [21]:
with open("opus_output.md", "r") as f:
    content = f.read()

print(content[:1500])


# Årsredovisning 2022

## Innehåll

### ÖVERSIKT
- Det här är
- Vår historia .......................................................................................................6
- Året i korthet ..................................................................................................8
- Sammanfattande nyckeltal & KPI:er ................................................................10
- FlexQube VD har ordet ....................................................................................................12

### STRATEGI
- Marknad och Trender ......................................................................................18
- Strategisk Modell.............................................................................................20
- Strategisk Riktning ...........................................................................................22
- Kundbas .........................................................................................................

### Visa markdown med Colabs Markdown-rendering

In [20]:
from IPython.display import Markdown

with open("opus_output.md", "r") as f:
    content = f.read()

Markdown(content)


# Årsredovisning 2022

## Innehåll

### ÖVERSIKT
- Det här är
- Vår historia .......................................................................................................6
- Året i korthet ..................................................................................................8
- Sammanfattande nyckeltal & KPI:er ................................................................10
- FlexQube VD har ordet ....................................................................................................12

### STRATEGI
- Marknad och Trender ......................................................................................18
- Strategisk Modell.............................................................................................20
- Strategisk Riktning ...........................................................................................22
- Kundbas ..........................................................................................................24
- Innovation och Produktutveckling ...................................................................26
- Djupdykning/Produkt ......................................................................................30
- Djupdykning/Kundprojekt ...............................................................................34
- Produktion .......................................................................................................36
- FlexQubes Distributörer ..................................................................................38
- Aktiekapital, aktien och ägarförhållanden .......................................................39

### FÖRVALTNINGSBERÄTTELSE
- Allmänt om verksamheten ...............................................................................44
- Kommentar till 2022 års finansiella utveckling ................................................46
- Förväntningar om den framtida utvecklingen .................................................47

### FINANSIELLA RAPPORTER
- Koncernens resultaträkning .............................................................................54
- Koncernens balansräkning ............................................................................55
- Koncernens förändringar i eget kapital .........................................................57
- Koncernens kassaflödesanalys ......................................................................58
- Moderbolagets resultaträkning ......................................................................59
- Moderbolagets balansräkning ........................................................................60
- Moderbolagets förändringar i eget kapital .......................................................61
- Moderbolagets kassaflödesanalys ...................................................................62
- Noter ...............................................................................................................63
- Revisionsberättelse ..........................................................................................77

### HÅLLBARHET
- Globala målen för hållbar utveckling ...............................................................82
- Vår hållbarhetsstrategi .....................................................................................84
- Miljömässig hållbarhet.....................................................................................86
- Socialt ansvar ...................................................................................................94
- Styrning ...........................................................................................................102

### BOLAGSSTYRNING
- Ordförande har ordet ......................................................................................110
- Bolagsstyrningsrapport ...................................................................................116
- Styrelse ............................................................................................................125
- Ledande befattningshavare .............................................................................128
- Årsstämma och finansiell kalender ..................................................................133
- Kontakt ............................................................................................................133

---

## Om FlexQube

FlexQube är en global leverantör av modulära och robusta mekaniska vagnar och robotiserade lösningar för materialhantering. Koncernen grundades 2010 och har sedan dess säkrat ett stort antal framstående företag som kunder.

FlexQube är ett teknikbolag med huvudkontor i Göteborg samt egna verksamheter i USA, Mexiko, Tyskland och England. Bolaget är verksamt inom vagnsbaserad materialhantering genom ett patenterat modulkoncept. FlexQube utvecklar och designar kundanpassade lösningar för både robotiserad och mekaniserad vagnslogistik. Genom företagets egenutvecklade och unika automationskoncept erbjuds robusta och självkörande robotvagnar. FlexQube har över 1000 kunder i 37 länder där de primära marknaderna är Nordamerika och Europa.

FlexQubes kunder återfinns inom bland annat tillverkningsindustrin, distribution- och lagerverksamheter. Exempel på större kunder är Tesla, Amazon, Volvo Cars, Siemens, Autoliv, Scania, Oda och Mann+Hummel.

---

## Vår historia – Från mekaniskt till robotteknik

Historien om FlexQube startade för 30 år sedan i de centrala delarna av Sverige, med tre killar som utvecklade en kärlek för att leka med Lego®. De växte upp och började sina professionella karriärer, men kärleken till att kunna bygga ihop konstruktioner i ett till synes oändligt antal kombinationer försvann inte.

Samtidigt som han arbetade för en global tillverkare av anläggningsmaskiner märkte Per Augustsson (CTO, FlexQube) att materialhanteringen inte uppfyllde moderna krav. Industrins normer för svetsade lösningar var inte anpassningsbara nog för att hantera den ständiga förändringen som industrin genomgår.

### Tidslinje

**2010** - FlexQube startas av Anders, Christian och Per i december 2010.

**2011** - Patent beviljas.

**2012** - Försäljningsstart.

**2013** - FlexQube Inc. startas.

**2014** - "Made in USA".

**2015/2016** - Vinnare på "33-listan" två år i rad och ny logotyp.

**2017** - Försäljning till 22 länder och notering på Nasdaq First North.

**2018** - Försäljning till totalt 25 länder och tillväxt med ca 140 procent. FlexQube GmbH startas och genomför riktad nyemission om 62,7 MSEK.

**2019** - Expansion till UK, samarbete med LR Intralogistik GmbH och lansering av eQart®. Flytt av huvudkontor i Sverige.

**2020** - Flytt av huvudkontor i USA. Inlett samarbete med 9 återförsäljare i sju olika länder. Ökad försäljning och vidareutveckling av eQart® line.

**2021** - Orderingångstillväxt på 98%.

**2022** - Ett erkänt robotteknikföretag

**2023** - Lansering av FlexQube Navigator AMR

**2024** - Orderingång robotkoncept överstiger 50 procent av total volym

---

## Året i korthet

### Nyckeltal Q1-Q4 2022

**Totala antalet kunder ackumulerat sedan 2010:** 1060

**Antalet länder som FlexQube sålt till:** 37

**Antalet nya kunder 2022:** 96

### Kvartalshöjdpunkter

#### Q1 - Mars:
- FlexQube Navigator AMR visas upp för första gången på Modex i Atlanta
- Erhåller order från Siemens Mobility värd 5 MSEK
- FlexQube erhåller fler ordrar från Tesla Gigafactory Berlin
- Bolaget erhåller eQart® Line order värd 5 MSEK

#### Q2 - April-Juni:
- FlexQube erhåller en uppföljningsorder värd 3,7 MSEK från befintlig kund i Nordamerika
- FlexQube Navigator AMR vinner pris på Robotics Innovation Award
- L'Oréal beställer eQart® Line från FlexQube värda 2,9 MSEK
- Amazon lägger order för ytterligare robotar och vagnar värda 15 MSEK
- FlexQube visar nytt rekordkvartal avseende orderingång (54,9 MSEK) och försäljning (56,1 MSEK)

#### Q3 - Juli-September:
- Internationell biltillverkare lägger order värd 5 MSEK
- Order på 8 eQart® Line robotvagnar till befintlig större kund
- FlexQube tar första order på eQart® Line till kund i Kina
- FlexQube visar nytt rekordkvartal avseende försäljning (56,4 MSEK) och är lönsamma på EBITDA-nivå för första gången sedan bolaget noterades

#### Q4 - November-December:
- FlexQube erhåller ordrar värda 6,4 MSEK i Mexiko
- Flyttar till nytt huvudkontor i Mölndal med dubblerad lokalyta
- FlexQube erhåller robotvagnsorder värd 3 MSEK
- FlexQube erhåller ordrar värda 3,6 MSEK till ett rymd- och försvarsbolag
- Mårten Frostne utsedd till ny VD på FlexQube
- FlexQube redovisar för första gången omsättning över 200 MSEK på ett helår

### Finansiella höjdpunkter 2022

- **Orderingången** ökade med **14,9 procent** till 178,4 MSEK (155,3). Rensat för valutakurspåverkan mellan jämförelseperioderna har orderingången ökat med 3 procent.

- **Nettoomsättningen** ökade med **81,7 procent** till 204,6 MSEK (112,6). Rensat för valutakurspåverkan mellan jämförelseperioderna har nettoomsättningen ökat med 65,4 procent.

- **Rörelseresultatet före avskrivningar (EBITDA)** uppgick till -1,5 MSEK (-17,4) och rörelseresultatet före finansiella poster (EBIT) uppgick till -6,4 MSEK (-22,0).

- **Resultat före skatt** uppgick till -7,0 MSEK (-22,5).

- **Resultat per aktie** uppgick till -0,9 SEK (-2,7).

- **Kassaflödet** uppgick till -5,6 MSEK (21,5), varav -22,0 MSEK (-29,3) från den löpande verksamheten, investeringsverksamheten med -8,1 MSEK (-2,8) och finansieringsverksamheten med 24,5 MSEK (53,7).

- **Likvida medel** uppgick till 30,5 MSEK (34,9) vid periodens utgång.

---

## Sammanfattande nyckeltal

### Flerårsöversikt och KPI:er

| Enhet | 2022 | 2021 | 2020 | 2019 | 2018 |
|-------|------|------|------|------|------|
| **Orderingång** (TSEK) | 178 428 | 155 282 | 78 369 | 81 208 | 63 743 |
| **Nettoomsättning** (TSEK) | 204 594 | 112 630 | 82 163 | 72 561 | 68 901 |
| **Försäljningstillväxt koncernen** (%) | 82% | 37% | 13% | 5% | 138% |
| **Rörelseresultat före avskrivningar (EBITDA)** (TSEK) | -1 514 | -17 362 | -15 303 | -20 522 | -5 971 |
| **Rörelseresultat (EBIT)** (TSEK) | -6 365 | -21 995 | -19 979 | -21 722 | -6 714 |
| **Rörelsemarginal** (%) | -3% | -20% | -24% | -30% | -10% |
| **Resultat före skatt** (TSEK) | -6 997 | -22 533 | -20 124 | -21 801 | -6 901 |
| **Vinstmarginal** (%) | -3% | -20% | -25% | -30% | -10% |
| **Resultat per aktie före och efter utspädning** (SEK) | -0,9 | -2,7 | -2,7 | -2,9 | -1,1 |

### Finansiell ställning
| **Soliditet** (%) | 45% | 55% | 56% | 74% | 81% |
| **Nettoskuld inklusive aktieägarlån** (TSEK) | -44 935 | -54 373 | -23 711 | -45 175 | -78 565 |
| **Rörelsekapital** (TSEK) | 73 135 | 39 350 | 27 547 | 32 989 | 24 496 |
| **Balanslikviditet** (%) | 168% | 225% | 240% | 311% | 502% |
| **Rörelsekapital i procent av nettoomsättning** (%) | 36% | 35% | 34% | 45% | 36% |
| **Kassalikviditet inklusive outnyttjad del av check** (%) | 102% | 163% | 170% | 205% | 406% |
| **Eget kapital per aktie före och efter utspädning** (SEK) | 8,4 | 8,8 | 5,7 | 8,9 | 11,7 |

### Kassaflöde
| **Kassaflöde från den löpande verksamheten** (TSEK) | -21 964 | -29 323 | -11 379 | -28 474 | -23 541 |
| **Kassaflöde från investeringsverksamheten** (TSEK) | -8 087 | -2 834 | -2 957 | -13 496 | -5 278 |
| **Kassaflöde från finansieringsverksamheten** (TSEK) | 24 451 | 53 678 | 9 778 | -316 | 54 781 |

### Aktier
| **Antal aktier vid periodens slut före och efter utspädning** (Tst) | 8 233 | 8 233 | 7 433 | 7 433 | 7 433 |
| **Genomsnittligt antal aktier före och efter utspädning** (Tst) | 8 233 | 8 233 | 7 433 | 7 433 | 6 385 |

### Anställda
| **Medelantal anställda** (st) | 60 | 39 | 30 | 24 | 17 |
| **Antal anställda vid periodens slut** (st) | 58 | 44 | 36 | 32 | 21 |

### Orderingång per marknad 2022
- **Nordamerika:** 60%
- **Mexiko:** 19%
- **DACH*** (Tyskland, Österrike och Schweiz): 10%
- **Övriga Europa:** 7%
- **UK:** 4%

---

## VD HAR ORDET

### Ett rekordår med viktiga och stora milstolpar

**KÄRA AKTIEÄGARE,**

# Vårt mycket starka 2021

Vårt fokus har från starten varit att sikta på de mest framstående företagen inom respektive bransch med stort fokus på tillverkning. Bakom de 1000 kunderna jag nämnde tidigare ligger hundratals välkända, globala och framgångsrika företag inom sina respektive verksamhetsområden som alla valt att arbeta med toppmoderna materialhanteringsvagnar och robotar från FlexQube. Dessa kunder är kärnan i vår verksamhet och den verkliga inspirationen bakom våra innovativa produkter.

Med cirka 100 procent i orderingångstillväxt kunde vi gå in i 2022 med en stabil orderbok och fortsätta att jobba i linje med vår strategi för att bli ett erkänt robotföretag samt ytterligare accelerera försäljningen. Jag är glad att kunna avsluta ett mycket framgångsrikt år för samtliga produktområden på alla våra huvudmarknader. Från starten 2010 har vi nu kommit att betjäna mer än 1000 kunder i totalt 37 länder. Fantastiskt!

Behovet av våra lösningar växer, speciellt när vi nu ser automatiseringstrenden ta fart för materialhantering inom tillverkande industri. Eftersom robotkonceptet kommer att bli en allt mer betydande del av vår produktportfölj under de kommande åren är jag fortfarande mycket nöjd över att se att vårt mekaniska koncept också visade en solid tillväxt på 76 procent under 2022. Kundresan börjar ofta med mekaniska vagnar och de är en viktig del av alla företags logistikprocesser inom tillverkning eller distribution. Robotik och automatisering är fortfarande en ovanlig företeelse inom våra kunders internlogistikprocesser, men utsikterna för det kommande decenniet ser mycket positiva ut.

Att arbeta med denna typ av företag bidrar på många sätt till långsiktig utveckling för FlexQube som bolag. För det första blir vi utmanade av de mest krävande kunderna på marknaden i att hitta optimala lösningar för deras processer och produkter. För det andra skapar vi en stark pool av referensprojekt som kommer att öka vår trovärdighet hos andra potentiella kunder. För det tredje anställer dessa företag duktiga medarbetare som utvecklar sina karriärer snabbt, och inte sällan går vidare till nya verksamheter inom relativt kort tid, där de sedan tar in oss för andra eller tredje gången. Dessa områden tillsammans med många andra positiva aspekter skapar ringar på vattnet som hjälper oss att växa långsiktigt.

Under 2022 fortsatte vi att nå nya milstolpar för vårt AGV-system som kallas eQart® Line. Orderingången fortsatte att öka under hela året med en rekordnivå för orderingången under fjärde kvartalet. Detta var vår första automationsprodukt vi utvecklade och började sälja, och vi har lärt oss mycket på vägen. Automationsprodukter och projekt kräver mer tid från start till mål, både internt men likväl som i försäljningsprocessen hos våra kunder. Därför kan jag fortfarande konstatera att vi bara är i början av vår försäljningsresa för vårt AGV-system, samtidigt som vi lanserar en ny robot som heter FlexQube Navigator AMR i vår nya produktlinje AMR-system. Friheten att skapa skräddarsydda AGV:er med vårt modulära och mekaniska vagnkoncept som bas är mycket attraktiv för våra kunder. Det koncentrerar fokus till materialpresentationen istället för materialtransporten som också är vad vi kallar för användarvänlig automation. Enkel att förstå, lätt att installera och snabb leveranstid. Ett unikt värdeerbjudande på en annars icke diversifierad marknad med många företag som erbjuder mer eller mindre samma produkter.

I juni 2023 kommer jag att lämna över VD rollen till vår nya VD Mårten Frostne. Jag är enormt stolt över mina 11 år som VD, alla fantastiska medarbetare och rekordåret 2022. Timingen känns helt rätt för en ny erfaren ledare som tar oss mot nya väl definierade mål och jag ser fram emot att vara del av teamet med fokus på våra stora kunder och affärsutveckling. Vi ska fortsätta på den inslagna vägen för FlexQube med fokus på automation och att bygga ett starkt globalt varumärke inom internlogistik. Vi har den stora resan för våra robotvagnar framför oss och jag känner mig väldigt trygg med att Mårten är rätt person att leda oss framåt.

I år fortsatte vi att göra betydande framsteg mot vårt mål att vara en ledande leverantör av innovativa och automatiserade materialhanteringslösningar. Automatisering är inte bara ett sätt att minska kostnader eller förbättra effektivitet; det är en väsentlig del för våra kunder att förbli konkurrenskraftiga och växa sin verksamhet. Om man tittar på diversifieringen av vår kundbas och spridningen av det geografiska fotavtrycket är tillväxtmöjligheterna betydande. Marknaden för materialhanteringsvagnar såväl som för robotar är fortfarande mycket fragmenterad och vi kan med självförtroende säga att vi vill bli marknadsledande för båda dessa. Vi jobbar fortsatt engagerat i linje med vår strategi och ser med stor tillförsikt på de möjligheter som ligger framför oss.

Tack för ditt fortsatta stöd!

**Anders Fogelberg**  
VD FlexQube AB (publ)

> "I år fortsatte vi att göra betydande framsteg mot vårt mål att vara en ledande leverantör av innovativa och automatiserade materialhanteringslösningar"  
> *Anders Fogelberg, VD*

---

# Strategi

## Marknad och Trender

Våra kunders stora fokus på både effektivitet och flexibilitet ger oss en idealisk position som leverantör av applikationsspecifika vagn- och robotlösningar för kunder på en stor diversifierad marknad med ett närmast ändlöst antal kravbilder.

Som en global leverantör av modulära och robusta industrivagnar och robotar har FlexQube hjälpt kunder i över tio år att lösa sina interna logistikproblem. Under den här tiden har det gett oss stor insikt i förändringarna inom branschen och våra kunders behov.

De krav som ställs på FlexQubes kunder att tillverka anpassade och personifierade produkter är mycket stora. Utmaningarna är som störst inom tillverkningsindustrin, där företag väljer att tillverka många olika produkter på samma monteringslina för att uppnå en hög grad av flexibilitet att snabbt kunna anpassa sin tillverkning efter kundernas efterfrågan. Pandemin förvärrade detta ytterligare, genom att lägga till brist på arbetskraft och problem med försörjningskedjor till en redan komplex miljö.

Alla de stora tillverkarna skapar sina egna varianter av framtidens fabrik för att kunna hantera de stora utmaningarna som nämns ovan. BMW har sin "iFactory", Audi har sin "360 factory" och Hyundai-Kia har sin "E-FOREST". De gemensamma nämnarna för alla är ett stort fokus på flexibilitet, effektivitet och hållbarhet.

Monteringslinor med många olika produkter ställer stora krav på materialhanteringen, där en stor mängd artiklar, med hög leveransfrekvens, behöver transporteras från lagret till montörerna som bygger produkterna, till exempel en bil. Den höga leveransfrekvensen av material skapar ett stort behov av att automatisera materialtransporter från lager till monteringslinjerna. Marknaden för materialhanteringsrobotar (AGV och AMR) förväntas växa till 14 miljarder USD år 2025, som motsvarar en CAGR på 20 procent.

Implementeringen av smarta tillverkningsprocesser för transport och presentation av material inom fabriker är dock svårare än vad många tror, även om utvecklingen går snabbt framåt. Mognadsgraden hos våra kunder att implementera automation varierar stort, men FlexQube har en fördel att kunna erbjuda kunden ett manuellt vagnsystem som senare kan vara en del av ett AGV- eller AMR-system.

En annan stor trend inom tillverkningssektorn är den ökande efterfrågan på hållbara produkter. Konsumenter efterfrågar i allt högre grad produkter som är tillverkade på ett hållbart sätt, med tonvikt på att minska avfall, energiförbrukning och utsläpp. Detta har lett till en ökning av antalet företag som fokuserar på hållbarhetsinitiativ och utveckling av mer hållbara tillverkningsprocesser.

Våra kunder börjar fråga sina leverantörer om hållbarhetsfördelarna med deras produkter. Även om dessa frågor blir vanligare är de fortfarande inte vanliga. Men internt förbereder vi oss genom att förbereda forskning om hållbarhetseffekterna av FlexQube-systemet till våra vagnar och robotar genom C02-studier, underhålls- och reservdelsavtal och produktlivscykelanalyser. Det här är en extremt spännande tid att vara en del av denna resa inom tillverkning och en tid som vi är glada över att dela tillsammans med våra kunder.

På FlexQube är vi fast beslutna att ge våra kunder det bästa inom materialhantering och robotteknik. Våra produkter är designade för att ge högsta nivå av effektivitet och kvalitetskontroll, vilket hjälper företag att förbli konkurrenskraftiga på marknaden och öka sin lönsamhet. Med vår expertis och erfarenhet inom tillverkningsindustrin är vi väl positionerade för att hjälpa våra kunder att behålla en konkurrensfördel inom sina branscher.

### Trender

**SMART TILLVERKNING**
Avancerad teknik som artificiell intelligens, robotik och Internet of Things för att automatisera tillverkningsprocesser.

**VARIATION I KUNDKRAV**
En storlek passar inte alla då fler kunder söker större variation för sina lösningar. Avancerad teknik som artificiell intelligens, robotik och Internet of Things för att automatisera tillverkningsprocesser.

**HÅLLBARA PRODUKTER**
Hållbara initiativ blir viktigare bland kunderna.

---

## Strategisk Modell

Vår strategi bygger på fyra komponenter: vision, mission, strategiska mål och kärnvärden. Visionen utgör vår långsiktiga målsättning och missionen ger oss möjlighet att förstå varför vi finns till. Strategiska mål definierar de områden vi ska fokusera på för att uppnå vår vision. Slutligen hjälper våra kärnvärden oss att förstå hur vi ska uppnå våra mål.

### VARFÖR
Vi tror att vi är en gamechanger och gör våra kunder världsledande inom internlogistik, med full flexibilitet att anpassa produktionen kontinuerligt, år efter år, med minimal påverkan på produktionen och högsta hållbarhet.

### VISION
Vi flyttar material i varje fabrik och lager i världen

### MISSION
Vi existerar för att göra kunder världsledande i internlogitik

### HUR
Vi utvecklar och anpassar ständigt vårt patenterade modulära koncept för bästa utnyttjande och effektivitet. Lätt att designa, bygga om och automatisera för ideala internlogistiska lösningar.

### SYFTE
Bygga världsledare i intralogistik

### VAD
Vi erbjuder modulbaserade och standardiserade byggblock för att skapa framtidssäkrade och hållbara interna logistiklösningar. Tillsammans med våra kunskapsdatabas, skräddarsydda lösningar och omfattande Solutions Library™, skapa bästa möjligheter att göra våra kunder världsledande inom intern logistik.

### STRATEGISKA OMRÅDEN
- Stärka arbetet med befintliga kunder
- Accelerera robotförsäljningen
- Öka effektivitet i operations
- Lönsamhet och kassaflödesoptimering
- Utveckla personal och företagskultur
- Service och eftermarknad

### KÄRNVÄRDEN
- Mod
- Kundfokus
- Förtroende
- Branchledande

---

## Strategisk Riktning

Bolaget befinner sig i en spännande fas med hög tillväxt vilket genomsyrar koncernens huvudsakliga strategiska områden för de kommande åren. Lansering av FlexQube Navigator AMR är huvudmålet för 2023 och tillsammans med den fortsatta utrullningen av eQart® Line, siktar vi på att bli en marknadsledande leverantör av materialhanteringsvagnar och robotar för internlogistik. Fortsatt utveckling av produktkoncept, kunderbjudande, marginaler och våra anställda är alla viktiga faktorer i vägen framåt.

### Våra 6 strategiska områden

1. **Stärka arbetet med befintliga kunder**
2. **Accelerera robotförsäljningen**
3. **Öka effektivitet i operations**
4. **Lönsamhet & kassaflödesoptimering**
5. **Utveckla personal och företagskultur**
6. **Service & eftermarknad**

### Vår resa fortsätter…

**01 Mekanisk fas** (2010–2017)  
**02 Transformationsfas** (2018–2020)  
**03 Robotteknikfas** (2021–2026)

---

## Målsättningar

### På kort sikt är FlexQubes mål att:

- Fortsätta ett mycket aktivt försäljningsarbete för att driva tillväxt och öka marknadsandelarna på alla relevanta marknader, primärt inom robotförsäljning.
- Utöka säljorganisation samt stärka kund- och försäljningssamarbetet med olika partners för primärt robotkonceptet och fortsätta utveckla interna försäljningsprocesser för att öka konverteringen.
- Säkerställa fullt fokus i hela organisationen för lansering av FlexQube Navigator AMR samt fortsätta utrullning av eQart® Line. I arbetet ingår även att stärka samarbetet med partners såsom integratörer och återförsäljare för att få ett optimerat erbjudande gentemot våra kunder.
- Bredda kundbasen för att uppnå ytterligare ökad spridning till andra brancher på nuvarande koncept, men högt fokus på befintliga stora inarbetade kundgrupper för bearbetning avseende robotkonceptet, i synnerhet FlexQube Navigator AMR.
- Lansera helt ny organisation för service- och eftermarknad i både USA och Sverige med fokus på både robotinstallationer men även service för det mekaniska konceptet där bolaget nu har en stor aktiv vagnpark på alla marknader.
- Förstärka marginal och kassaflöde genom högt fokus på robotförsäljning, implementera service- och eftermarknadsorganisationen samt optimering av Supply Chain.

### På medellång sikt är FlexQubes mål att:

- Fortsätta den kraftiga ökningen av marknadsandelar på huvudmarknaderna Nordamerika, Mexiko, Tyskland och England. I synnerhet avseende robotförsäljning.
- Stärka vår marknadsposition genom fortsatt breddning och expansion av kundbasen samt återkommande försäljning till befintliga kunder, genom att nå ut till och etablera oss på nya marknader.
- Attrahera, utveckla och behålla talanger för att bygga ett starkt team för framtiden. En konkurrenskraftig och coachande företagskultur. Främja framtida ledare inom varje område av företaget för att säkra en organisation som kan hantera framtida tillväxt.

### Långsiktigt har FlexQube ambitionen att:

- Bli marknadsledande leverantör av materialhanteringsvagnar och robotar för internlogistik. FlexQube ska vara den bästa lösningen för morgondagens produktions-, lager- och distributionslogistik samt göra sina kunder världsledande inom intralogistik. En nödvändig förutsättning för detta är att ha närvaro och infrastruktur – inom försäljning, tillverkning och distribution – i alla relevanta delar av världen.

---

## Med kunden i fokus

FlexQubes kunder återfinns i olika segment och regioner

# Modulärt koncept

FlexQubes koncept erbjuder standardiserade byggblock som är specifikt utvecklade med materialhanteringsvagnar och robotar i åtanke. Detta säkerställer att funktion och prestanda blir optimerad för den tuffa miljö som vagnarna utsätts för.

## Autonoma vagnar

FlexQubes erbjudande inom autonoma lösningar tar det modulära konceptet till nästa nivå och skapar en helt skräddarsydd automationslösning, där kunderna kan kombinera standardiserade byggblock med intelligenta funktioner för att uppnå optimal effektivitet och flexibilitet i sina verksamheter. FlexQubes AGV-system passar mer standardiserade automationsscenarion medan vår FlexQube AMR-system kan programmeras och styras in i minsta detalj. Båda systemen stöder det modulära konceptet fullt ut och lastbäraren kan fysiskt konfigureras i ett närmast obegränsat olika konfigurationer utifrån kundens behov.

Genom att integrera våra AGV- eller AMR-system i sina arbetsflöden kan företag anpassa sina lösningar efter specifika behov och snabbt anpassa sig till förändrade krav eller arbetsförhållanden. Detta möjliggör en smidig och kostnadseffektiv omställning och främjar en mer hållbar och konkurrenskraftig produktion.

## Designstandard

FlexQubes vagnar och robotar designas enligt en steg-för-steg process med standardiserade val i varje delmoment. Detta gör det enkelt att utveckla kvalitetssäkrade lösningar, även om alla vagnar som skapas ser olika ut och har olika funktion.

## Expert på materialhantering

Eftersom FlexQube arbetar med många olika kunder inom olika industrier och regioner byggs en stor kunskapsbank upp. FlexQube samlar alla lösningar i SolutionsLibrary™, som är tillgängligt för alla kunder via FlexQubes hemsida. Ju fler lösningar som genereras, desto större blir kunskapsbanken och desto större sannolikhet att hitta en redan designad vagn och robot för att lösa ett kundbehov. Utöver SolutionsLibrary™ har FlexQube en hög kunskapsnivå kring logistikutveckling, strategierna för detta och vilka framtidens trender är.

## Skalbart och globalt koncept

Tack vare FlexQubes standardiserade byggblock och standardiserade designprocess kan försäljning snabbt etableras på nya marknader. En design kan skapas till kund oavsett geografisk placering och konceptet möjliggör att skicka FlexQubes produkter omonterade för snabb och billig transport. Även tillverkning och montering kan snabbt etableras vid behov.

## Enkel integration

Framtidens fabriker kräver samspel mellan olika processer och utrustningar för att skapa effektiva materialflöden. FlexQube kan tack vare sin flexibilitet enkelt skapa vagnar och robotar som kan integreras med utrustning från tredje part. Detta gäller framförallt dragfordon, mor-/dottertåg och lösningar för automatiska materialrobotar, vilka är de vanligaste integrationerna.

# Kundbas

Per den 31 december 2022 har FlexQube sålt till över 1000 kunder fördelade på 37 olika länder. Vi ser fler och fler förfrågningar från kunder inom nya affärssegment. Ett exempel på ett växande segment är distribution av livsmedel, som varit mycket starkt under 2022. För FlexQube som företag strävar vi efter att finnas i flera segment för att minska exponeringen mot enskilda segment och dela kunskap mellan segmenten för att hjälpa våra kunder att bli mer konkurrenskraftiga i en utmanande marknad.

## Segment där FlexQube är verksamma:

- Tillverkning av kommersiella bilindustrin
- Underleverantörer till fordon såsom lastbilar, bussar och tåg
- Tillverkning av bilar
- Tillverkning av vindkraftverk och andra energirelaterade produkter
- Tillverkning av anläggnings- och industrimaskiner
- Tillverkning av försvarmaterial
- Tillverkning av vitvaror och elektronik
- Tillverkning av medicinsk utrustning
- Lager och distribution av varor

## Information om orderingång:

Orderingången för helåret 2022 ökade med 14,9 procent jämfört med 2021 till 178,4 MSEK. Under andra kvartalet 2022 gjorde koncernen en ny rekordnotering avseende orderingång på ett enskilt kvartal med 56,4 MSEK.

**Antal nya kunder 2022:** 96

# Innovation och Produktutveckling

Innovation och produktutveckling är ett viktigt område för FlexQube för att hjälpa kunder med deras stora utmaningar inom materialhantering. Utmaningarna är som störst inom tillverkningsindustrin, där företag väljer att tillverka många olika produkter på samma monteringslina för att uppnå en hög grad av flexibilitet att snabbt kunna anpassa sin tillverkning efter kundernas efterfrågan.

Monteringslinor med många olika produkter ställer stora krav på materialhanteringen, där en stor mängd artiklar, med hög leveransfrekvens, behöver transporteras från lagret till montörerna som bygger produkterna, till exempel en bil.

FlexQube-konceptet hjälper kunderna att både effektivisera transporterna av materialet, samt att skapa ergonomisk, kostnadseffektiv och säker materialpresentation för montörerna.

FlexQube fortsätter att utveckla innovativa produkter, förenklat utvecklingen av produktsegmenten genom att byta namn på våra manuella och autonoma lösningar. Vi utvecklar och säljer tre produktfamiljer med ett stort antal olika varianter som tillsammans ger kunden ett unikt och komplett system för materialpresentation och -transport.

## Produktfamiljer:

- **Vagnsystem** - Mekaniska vagnar och LiftRunner tågvagnslösningar
- **AGV-system** - eQart® Line
- **AMR-system** - FlexQube Navigator AMR

## Utvecklingsfokus

FlexQubes utveckling har en stor bredd, och vi fokuserar vår utveckling inom respektive produkt där det skapar mest värde för våra kunder.

### Vagnssystem
För vårt vagnssystem fokuserar vårt sälj- och designteam mycket på de applikationer vi skapar och hur kan vi kombinera våra mekaniska byggblock för att lösa kundernas utmaningar gällande transport och materialpresentation. Även de mekaniska byggblocken optimeras utifrån tillverkning och monteringsbarhet. Vi utvärderar flexibla koncept för tillverkning som möjliggör större grad av kundorderstyrd tillverkning med minskade lager och snabbare leveranstider som följd.

### AGV och AMR-system
Automated Guided Vehicles (AGV) och Autonomous Mobile Robots (AMR) är båda typer av mobila robotar som används för att transportera material och gods inom produktionsanläggningar, lager och distributionscenter. AGV-system följer förutbestämda rutter och är beroende av fasta guidningssystem, vilket passar bäst till standardiserade rutter. AMR-system navigerar autonomt med mer avancerade sensorer och mjukvara, vilket gör dem mer anpassningsbara och lämpade för dynamiska arbetsmiljöer.

För vårt linjeföljande AGV-system utvecklas teknik och byggmoduler in-house. Det innefattar allt från design och tillverkning av kretskort, utveckling av python-mjukvara för kamerabaserad navigation, till utveckling av Android som kunden använder för att interagera med roboten. Egen teknikutveckling är nödvändig för att kunna uppnå vårt mål, att erbjuda världens enklaste AGV-system som kan implementeras själv av kunden och säljas direkt till kund utan integratörer eller mellanhänder.

För vårt autonoma AMR-system finns det mycket färdigutvecklad teknik som uppfyller de krav vi har. Vi har valt att inte lägga all teknikutveckling hos oss själva och istället etablerat partnerskap med det absolut främsta företagen inom respektive teknikområde. T.ex. har vi partnerskap med BlueBotics kring autonom navigation, och vi samarbetar med Wiferion kring batterier och trådlös laddning. Genom att använda redan utvecklad och beprövad teknik kan vi ha ett stort fokus på hur roboten och applikationen ska utformas, med kundnyttan som högsta prioritet. Våra tekniksamarbeten har möjliggjort att vi på en kort tid kunnat ta fram världens första "icke lastbärande" AMR som ger helt nya möjligheter får våra kunder där en och samma AMR kan flytta en stor variation av olika lastbärare.

# FlexQube AMR-System: Robotics that delivers

Fokusmarknaden för FlexQube Navigator AMR ligger inom tillverkning eftersom det finns ett stort glapp mellan kunders egentliga behov och vad som finns tillgängligt på automationsmarknaden idag. Många företag har automatisering högt upp på agendan, men implementeringen av robotar inom logistik har knappt börjat, och det finns unika utmaningar relaterade till tillverkning som måste lösas.

För att förenkla implementering av automation i fabriker, lanserar vi vårt AMR-system som är nästa steg i FlexQubes automationsresa. Ett modulärt robotkoncept där en standardiserad robot kan flytta många olika typer av motoriserade lastbärare.

Utveckling av FlexQubes AMR-system är ett viktigt steg mot FlexQubes långsiktiga vision där de flesta vagnar i en fabrik är flexibla och självkörande.

FlexQube AMR-system minskar många av de utmaningar som är relaterade till automatisering inom tillverkning. Vi kallar det för "Robotics that delivers", som indikerar två tydliga mål med systemet.

## Leverera material till människor
För det första är fokus att "leverera" material till människor som monterar produkter, där ergonomi, effektivitet och säkerhet blir extra viktigt.

## Leverera som det lovar
För det andra, ett system som levererar som det lovar, dvs. att det gör det enkelt för kunden att installera och använda AMR-systemet.

## FlexQubes AMR-System består av:

- **FlexQube Navigator AMR** - Standardiserad robot för att navigera motoriserade lastbärare
- **FlexQubePLAY Coupling** - Smart och standardiserat interface mellan robot och motoriserad lastbärare
- **FlexQube load carriers** - Motoriserad lastbärare konfigurerad för att transportera specifikt material för ett "user case" i fabriken, t.ex. flytta säten, hjul eller instrumentbrädor

## Kärnvärden för AMR-systemet

Kärnvärdena för AMR-systemet är en viktig grundsten som sammanfattar värdegrunden för systemet samt är drivkraften för nya utvecklingsmöjligheter. De etablerade kärnvärden för systemet är baserat på fyra pelare: Smart, Säker, Skalbar och Successful.

### Smart
Världens första icke lastbärande robot möjliggör att en generisk AMR kan flytta många olika typer av lastbärare med varierande storlek och form. Detta möjliggörs genom ett standardiserad koppling som vi kallar FlexQubePLAY. Kopplingen möjligör ett ecosystem där även partners till FlexQube kan utveckla olika typer av lastbärare för olika applikationer i en fabrik.

### Säker
Genom det smarta interfacet kan FlexQube Navigator AMR identifiera hur stor lastbäraren är, och anpassa säkerhetsystemets laserzoner. Detta möjligör att lastbärarna kan variera i storlek, med bibehållen säkerhet. Det smarta interfacet möjligör också att lastbäraren kan utrustas med nödstopp, vilket är unikt i branchen.

### Skalbar
Det standardiserade kopplingsinterfacet gör det enkelt att lägga till fler robotar och lastbärare till systemet. Integration mellan AMR och lastbärare är redan utvecklat och testat av FlexQube, vilket gör det enkelt för kunden att skala upp systemet. Dessutom kan lastbärarna anpassas och byggas om när fabriken eller produkterna behöver förändras, som ställer krav på en annan materialpresentation.

### Successful
Fokus är att arbeta med projekt där AMR-systemet skapar ett stort värde för kunden. Det innebär att de som arbetar med och omkring roboten ska känna sig säkra och uppleva roboten som ett hjälpmedel snarare än ett hot. Det innebär att vi kan påvisa fabriksledningen en avsevärd effektivitetsförbättring, och det innebär att de som ska plocka material vid linan har den bästa ergonomin.

## Lastbärare - anpassningsbarhet i fem dimensioner:

1. **Funktion** - Plattformen byggs enligt FlexQubes modulära och flexibla koncept vilket gör att den helt anpassas efter kundens behov
2. **Storlek** - Variationen av storlek på våra lastbärare möjliggör stor skalbarhet och enkel anpassning till olika fabriker och processer
3. **Förvaring** - Toppstrukturen av våra lastbärare kan enkelt anpassas. Många av våra kunder transporterar material av stor variation och toppstrukturen kan justeras efter deras behov
4. **Vikt** - Plattformen och hållfastheten anpassas efter den vikt kunden behöver kunna transportera
5. **Rörelse** - Våra lastbärare har hög rörelsefunktionalitet, vilket möjliggör att de flexibelt kan transportera material i olika riktningar

# Kundprojekt: Signify eliminerar fotgängartrafik med automation

## Utmaning
Signify sökte efter ett sätt att eliminera mycket av de transporter som sker till fots på lager och tillverkningsgolvet. Detta inkluderade bland annat processen för operatörer att lämna sin position för att transportera delar till slutdestination, eller att flytta material från det externa till det interna lagret. Den övergripande utmaningen var att ta bort så mycket av den enskilda deltransporten som möjligt.

## Lösning
Signify började genom att hyra två AGV-system, två tuggervagnar och två hyllplansvagnar. Våra eQart® Line-robotar erbjöd Signify en automatiserad lösning som kunde hjälpa till att eliminera fotgängstrafiken. Lösningen säkerställde att de kunde transportera större volymer vagnar under samma tillfälle. Slutresultatet gav en positiv effekt på det ursprungliga problemet då de anställda kunde prioritera sin tid på andra arbetsuppgifter.

> "I genomsnitt innebär det ungefär 10-15 minuters besparingar per dag för varje genomförd transport. Då vi har tio stycken transportlinjer betyder det att vi kan spara upp till två och en halv timmar. När man börjar lägga ihop alla minuter under loppet av en vecka inser man hur en operatörs expertis inte utnyttjas till fullo."
> 
> *- Kyle Sturhis, Ingenjörschef Signify i Pennsylvania, USA.*

# Produktion

FlexQube tog under 2020-2021 viktiga strategiska steg framåt genom att börja producera i egna fabriker. Vi har nu produktion i Mölndal som servar den europeiska marknaden och i Duncan, South Carolina, som levererar till hela Nordamerika och Mexico. Tidigare hade FlexQube outsourcat sin produktion, men med den nya förändringen har vi kunnat förbättra vår produktkvalitet och leveranssäkerhet.

Med lager, montering och distribution i egna fabriker har vi möjlighet att öka kvaliteten på våra materialhantering- och automationslösningar. Tidigare var det svårt att ha full kontroll över tillverkningsprocessen och säkerställa att varje produkt uppfyllde högsta standard. Nu har vi fullständig insyn i varje steg av produktionsprocessen och kan säkerställa att varje produkt är tillverkad enligt specifikationerna och lever upp till högsta kvalitetsstandard.

FlexQubes produktion i egna fabriker har också möjliggjort testning av nya koncept på ett enklare och effektivare sätt. Vi kan nu experimentera med olika material och konstruktioner och snabbt se hur de fungerar i praktiken.

Med fullständig kontroll över tillverkningsprocessen kan vi snabbt göra justeringar på konstruktioner och optimera produkterna för att möta kundernas behov. Med högre kontroll över produktionsprocessen kan FlexQube nu även leverera produkter snabbare än någonsin tidigare. Detta är en stor fördel för företag som behöver snabba och pålitliga materialhanteringslösningar.

Fördelarna med produktion i egna fabriker begränsas inte bara till kvalitet och snabbhet. Vi har också möjlighet att effektivisera vår produktionsprocess genom att optimera flödet av material och minska avfall som bidrar till en ökad hållbarhet.

Genom att ha fullständig kontroll över produktionen kan FlexQube också se till att arbetarna är säkra och att arbetsförhållandena är goda. Under 2022 startade vi upp en satellitlokal för montering i Polen enligt samma koncept och processer som våra befintliga produktionslokaler i Sverige och USA. Detta möjliggör snabbare leveranser till centrala Europa och ökar vår totala produktionskapacitet till den europeiska marknaden.

Arbetet har fallit väl ut och vi fortsätter optimera samarbetet under 2023 genom att även involvera dessa resurser för service ute hos våra kunder, vilket vi tror är en stor tillväxtmöjlighet för koncernen. Under 2023 är vårt fokus att fortsätta utveckla våra interna processer och investera mer i form av utökade resurser för både Supply Chain och servicekoncept för våra kunder.

Sammanfattningsvis har FlexQubes satsning på produktion i egna fabriker lett till ökad kvalitet på produkterna, möjlighet att testa nya koncept enklare, samt möjlighet att göra snabba justeringar på konstruktioner för att optimera produkterna. Produktion i egna fabriker är en stor fördel för både FlexQube och våra kunder, och kommer säkerställa att företaget fortsätter att vara en ledande aktör inom materialhanterings- och automationslösningar i framtiden.

# FlexQubes Distributörer

Sedan 2020 har antalet distributörer växt. Vi säljer FlexQubes koncept genom totalt 16 internationella distributörer i 9 olika länder.

Vi erbjuder våra distributörer ett unikt koncept med vårt vagn- och robotsystem som i sin tur ger dem möjlighet till tillväxt, utöka sin kundbas och ge dem ett första steg inom industriautomatisering med våra AMR- och AGV-System. Vi har sett betydande framsteg i både Europa och Nordamerika i vår distributörsutveckling.

## NORDAMERIKA
- **UTAH**: SynerTech Automation, LLC
- **MINNESOTA**: Process Logic
- **GEORGIA**: McGee Storage & Handling, Air Specialists
- **WISCONSIN**: Wolter Group LLC
- **MEXICO**: Inter Price Logística
- **NORTH CAROLINA**: Carson Material Handling
- **TEXAS**: AGV America
- **KANADA**: RI-GO Lift Truck, Elevex

## EUROPA
- **POLEN**: IntraLogix
- **FRANKRIKE**: Ethic'Opex
- **SPANIEN**: DID Automation
- **ITALIEN**: BG Log
- **BELGIEN**: LeanFlow

## AFRIKA
- **SYDAFRIKA**: Lazar Robotic Welding

# Aktiekapital, aktien och ägarförhållanden

## FlexQube-aktien

Bolagets aktie är noterad på Nasdaq Stockholm First North under symbolen FLEXQ sedan 14 december 2017. FlexQube hade en omsättning under perioden 1 januari till 31 december 2022 om 1,2 miljoner aktier. Detta gav en genomsnittlig omsättning på ca 4 768 aktier per börsdag till ett värde av 290 534 SEK. Snittkurs för aktien under perioden var ca 61,7 SEK.

Senaste avslut vid periodens slut var 54,8 SEK, innebärande en uppgång om ca 83 procent från teckningskursen i samband med noteringen den 14 december 2017, eller en nedgång om ca 31 procent från stängningskursen den 31 december 2021.

## Aktiekapital

FlexQubes aktiekapital uppgår till 0,8 MSEK fördelat på 8 233 333 aktier. Enligt FlexQubes bolagsordning ska aktiekapitalet uppgå till lägst 0,5 MSEK och högst 2,0 MSEK och antalet aktier till lägst 5 000 000 och högst 20 000 000. Aktiernas kvotvärde är 0,1 SEK.

Aktierna i FlexQube är inte, och har inte varit, föremål för erbjudande till följd av budplikt, inlösenrätt eller lösningsskyldighet. Aktierna har heller inte varit föremål för något offentligt uppköpserbjudande. Aktierna har getts ut i enlighet med svensk lagstiftning och är denominerade i svenska kronor. Det finns inga inskränkningar i rätten att fritt överlåta aktier.

## Aktieägare i FlexQube

Per den 31 december 2022 hade FlexQube ca 1 900 aktieägare. Tabellen nedan visar bolagets största aktieägare per den 31 december 2022.

| Aktieägare | Aktier (T.) | Kapital och röster % |
|------------|-------------|---------------------|
| Christian Thiel genom Feldthusen Invest AB | 1 945 | 23.6% |
| Per Augustsson genom Augutech AB | 1 458 | 17.7% |
| Roosgruppen AB | 1 432 | 17.4% |
| Anders Fogelberg genom Birdmountain Invest AB | 946 | 11.5% |
| Nils-Robert Persson | 515 | 6.3% |
| FCG Fonder | 200 | 2.4% |
| Brofund Equity AB | 184 | 2.2% |
| Swedia Capital | 138 | 1.7% |
| Övriga | 1 415 | 17.2% |
| **Totalt** | **8 233** | **100%** |

## Central värdepappersförvaring

FlexQubes aktier är registrerade i ett avstämningsregister enligt lagen (1998:1479) om värdepapperscentraler och kontoföring av finansiella instrument. Kontoförande institut är Euroclear Sweden AB, Box 7822, 103 97 Stockholm. Inga aktiebrev har utfärdats för bolagets aktier. ISIN-koden för FlexQubes aktier är SE0010547075.

## Aktiekapitalets utveckling

Aktiekapitalet i FlexQube har sedan bolaget bildades i oktober 2012 förändrats i enlighet med vad som framgår av nedanstående tabell.

| År | Händelse | Förändring | Totalt | Förändring | Totalt |
|----|----------|------------|--------|------------|--------|
|    |          | Antal aktier (T) | Antal aktier (T) | Aktiekapital (TSEK) | Aktiekapital (TSEK) |
| 2012 | Nybildning | 50 | 50 | 50 | 50 |
| 2017 | Fondemission | - | 50 | 450 | 500 |
| 2017 | Aktiesplit 100:1 | 4 950 | 5 000 | - | 500 |
| 2017 | Nyemission | 1 333 | 6 333 | 133 | 633 |
| 2018 | Nyemission | 1 100 | 7 433 | 110 | 743 |
| 2021 | Nyemission | 800 | 8 233 | 80 | 823 |

## Konvertibler, teckningsoptioner med mera

FlexQube har per balansdag två aktiva teckningsoptionsprogram för styrelse, ledande befattningshavare samt övriga anställda inom bolaget.

FlexQube har totalt 201 850 teckningsoptioner utställda per balansdag 31 december 2022.

**Teckningsoptionsserie 2021-2024** löper på tre år och varje option ger rätt till teckning av en aktie till teckningskurs 56,43 kr. Totalt har detta program 84 000 teckningsoptioner utställda.

**Teckningsoptionsserie 2022-2025** löper på tre år och varje option ger rätt till teckning av en aktie till teckningskurs 109,6 kr. Totalt har detta program 117 850 teckningsoptioner utställda.

## Överenskommelser mellan nuvarande aktieägare

Såvitt styrelsen för FlexQube känner till finns det inte några överenskommelser eller motsvarande mellan aktieägare som syftar till gemensamt inflytande över FlexQube eller som senare kan leda till att kontrollen över FlexQube förändras.

## Incitamentsprogram

## Vissa rättigheter förenade med aktierna

FlexQube har endast ett aktieslag. Samtliga till aktien knutna rättigheter tillkommer den som är registrerad i den av Euroclear Sweden förda aktieboken. Rättigheterna förenade med aktier emitterade av bolaget, inklusive de som följer av bolagsordningen, kan endast ändras enligt de förfaranden som anges i aktiebolagslagen (2005:551).

### Rösträtt

Varje aktie berättigar innehavaren till en röst på bolagsstämmor. Varje aktieägare är berättigad till att rösta för det antal röster motsvarande innehavarens totala antal aktier i FlexQube.

### Rätt till utdelning och behållning vid likvidation

Aktierna ger lika rätt till andel i bolagets tillgångar, resultat och eventuellt överskott vid likvidation. Beslutar FlexQube att genom kontant- eller kvittningsemission ge ut nya aktier, teckningsoptioner eller konvertibler har aktieägarna företrädesrätt till teckning i förhållande till det antal aktier de förut äger.

Det finns dock inga bestämmelser i bolagets bolagsordning som begränsar möjligheten att, i enlighet med bestämmelserna i aktiebolagslagen, emittera nya aktier, teckningsoptioner eller konvertibler med avvikelse från aktieägarnas företrädesrätt.

## Utdelning och utdelningspolicy

FlexQubes strategi är fortsatt internationell expansion och att organiskt öka försäljningen kraftigt de närmaste tre till fem åren. I linje med bolagets strategi kommer tillväxt att prioriteras framför utdelning de närmsta åren, och framtida beslut om utdelning ska tas med hänsyn till FlexQubes utveckling och tillväxtmöjligheter.

Beslut om vinstutdelning fattas av bolagsstämman och utbetalas genom Euroclear Swedens försorg.

Utbetalning av vinstutdelning ska ske i SEK. Utdelning får endast ske med ett sådant belopp att det efter utdelningen finns full täckning för bolagets bundna egna kapital och endast om utdelningen framstår som försvarlig med hänsyn till:

- de krav som verksamhetens art, omfattning och risker ställer på storleken av det egna kapitalet; samt
- bolagets konsolideringsbehov, likviditet och ställning i övrigt (den så kallade försiktighetsregeln).

Rätt till utdelning tillkommer den som på den av bolagsstämman fastställda avstämningsdagen för utdelningen är registrerad som innehavare av aktier i den av Euroclear Sweden förda aktieboken.

Om aktieägare inte kan nås genom Euroclear Sweden kvarstår aktieägarens fordran på bolaget avseende utdelningsbeloppet och begränsas i tiden endast genom regler om tioårig preskription. Vid preskription tillfaller utdelningsbeloppet bolaget.

Varken aktiebolagslagen eller FlexQubes bolagsordning innehåller några restriktioner avseende rätt till utdelning för aktieägare utanför Sverige. Utöver eventuella begränsningar som följer av bank- eller clearingsystem i berörda jurisdiktioner, sker utbetalning till sådana aktieägare på samma sätt som till aktieägare med hemvist i Sverige.

# Förvaltningsberättelse

Styrelsen och verkställande direktören för FlexQube AB (publ), org. nr 556905-3944, med säte i Mölndal, avger härmed års- och koncernredovisning för räkenskapsåret 2022.

## Allmänt om verksamheten

FlexQube är ett teknikföretag som är specialiserat inom vagnsbaserad materialhantering genom ett patenterat modulkoncept.

Vår kärnkompetens är att utveckla och designa kundanpassade lösningar för både robotiserad och mekaniserad vagnslogistik. Vi är stolta över att kunna erbjuda självkörande robotvagnar, tack vare vårt egenutvecklade automationskoncept.

FlexQube är en global leverantör av modulära och robusta mekaniska vagnar och robotiserade lösningar för materialhantering. Vi grundades 2010 och med försäljningsstart under andra halvan av 2012, och har sedan dess fått ett stort antal prominenta företag som kunder.

Idag har FlexQube en säljorganisation som fokuserar på Europa och Nordamerika med tillverkning i Göteborg, Sverige för europamarknaden och i South Carolina, USA för den nordamerikanska och mexikanska marknaden.

FlexQube erbjuder lösningar för vagnsbaserad materialhantering, baserade på ett patenterat modulkoncept. Med ett ökande behov av anpassningsbara erbjudanden och högre förväntningar från konsumenter, har företaget utvecklat och designat kundanpassade lösningar för både robotiserad och mekaniserad vagnslogistik.

FlexQubes egenutvecklade och unika automationskoncept ger robusta självkörande robotvagnar som hanterar osäkerhet, snabba volym- och mixförändringar samt snabb teknisk utveckling. Med över 1000 kunder i 37 länder, främst i Nordamerika och Europa, erbjuder FlexQube konfigurerbara vagnar som möter kundernas behov av tillförlitliga och frekventa logistiklösningar.

Vårt mål är att hjälpa våra kunder att förbättra sin interna logistik och uppnå detta genom att skapa unika materialvagnar och robotar med modulära byggstenar, en innovativ designprocess och en hög kunskapsnivå av internlogistik. Vi strävar alltid efter att vara en ledande aktör på marknaden och vi ser fram emot att fortsätta växa och utveckla vår verksamhet.

### Globala kontor och verksamhet

- **Göteborg, Sverige**: HQ FlexQube-koncernen
- **Birmingham, England**: HQ FlexQube Ltd, Distributionscenter och tillverkning
- **Wroclaw, Polen**: Säljkontor, Distributionscenter
- **München, Tyskland**: FlexQube GmbH, Säljkontor
- **Duncan, USA**: HQ FlexQube Inc., Distributionscenter och tillverkning
- **Aguascalientes, Mexico**: Säljkontor, Lager
- **Indien**: Designkontor

## Anställda

Antalet anställda på FlexQube speglar den skalbara verksamhetsmodellen som koncernen aktivt arbetar med för att kunna nyttja stordriftsfördelar på längre sikt och samtidigt begränsa risken.

Antalet anställda per 31 december 2022 uppgick till 58 personer (44), varav 15 kvinnor (11). Medelantalet anställda under perioden januari till december 2022 uppgick till 60 personer (39), varav 15 kvinnor (9).

Det kan även noteras att i och med bolagets organisationsstruktur så förfogar bolaget över ytterligare ca 20-30 personer hos leverantörer och externa konsulter.

Trots att antalet anställda i bolaget är relativt få, förfogar bolaget ändå över en bred kompetens inom relevanta områden i och med de anställdas bakgrund, utbildning och erfarenheter.

Vidare anlitar bolaget vid behov de kompetenser som behövs samt i stor omfattning samarbetar med bolagets underleverantörer.

## Flerårsöversikt

För definitioner av nyckeltal, se not 1 på sidan 65.

| Resultat | Enhet | 2022 | 2021 | 2020 | 2019 | 2018 |
|----------|-------|------|------|------|------|------|
| Orderingång | TSEK | 178 428 | 155 282 | 78 369 | 81 208 | 63 743 |
| Nettoomsättning | TSEK | 204 594 | 112 630 | 82 163 | 72 | - |

# Finansiella nyckeltal

| Mått | Enhet | 2022 | 2021 | 2020 | 2019 | 2018 |
|------|------|------|------|------|------|------|
| Rörelseresultat före avskrivningar (EBITDA) | TSEK | -1 514 | -17 362 | -15 303 | -20 522 | -5 971 |
| Rörelseresultat (EBIT) | TSEK | -6 365 | -21 995 | -19 979 | -21 722 | -6 714 |

## Finansiell ställning

| Mått | Enhet | 2022 | 2021 | 2020 | 2019 | 2018 |
|------|------|------|------|------|------|------|
| Soliditet | % | 45% | 55% | 56% | 74% | 81% |
| Rörelsekapital | TSEK | 73 135 | 39 350 | 27 547 | 32 989 | 24 496 |
| Balanslikviditet | % | 168% | 225% | 240% | 311% | 502% |
| Kassalikviditet inklusive outnyttjad del av check | % | 102% | 163% | 170% | 205% | 406% |

# Förväntningar om den framtida utvecklingen

Vi förväntar oss en fortsatt förbättring av lönsamhet och kassaflöde genom att fortsätta expandera produktportföljen och öka försäljningen av befintliga produkter. Bolaget kommer att fortsätta investera i utveckling av nya produkter, som FlexQube Navigator AMR, för att möta marknadens behov. Det är dock viktigt att notera att FlexQube inte lämnar prognoser och att förväntningarna är baserade på den nuvarande kunskapen och marknadsförutsättningarna.

# Risker och osäkerhetsfaktorer

All affärsverksamhet innebär en viss risk, och det är därför viktigt att systematiskt bedöma och hantera risker. FlexQube arbetar kontinuerligt med att utvärdera och hantera sina risker genom att bedöma förebyggande åtgärder och att ha relevanta policyer och riktlinjer på plats. Moderbolaget FlexQube AB (publ) har en begränsad riskexponering vilket gör att beskrivningen främst hänförs till koncernen som helhet.

## Finansiella risker

FlexQube är utsatt för marknadsrisker och finansiella risker såsom valuta-, likviditets-, och kreditrisker. Det är FlexQubes styrelse som är ytterst ansvarig för hantering och uppföljning av koncernens finansiella risker. Valuta- och likviditetsrisken utgör de mest betydande finansiella riskerna medan ränte-, finansierings- samt kreditrisk kan tillmätas lägre risk.

Företaget arbetar aktivt med att identifiera och reducera riskerna genom olika åtgärder, såsom att säkerställa ett tillfredsställande kassaflöde och att samarbeta med långivare och finansiella samarbetspartners. Det finns en övergripande finanspolicy som syftar till att identifiera och minimera effekterna av de finansiella riskerna. Den praktiska hanteringen utförs av ekonomifunktionen i koncernen enligt finanspolicyn som fastställs av styrelsen varje år. Styrelsen erhåller löpande rapporter om bl.a. kassaflödet, skuldnivåer och resultatutveckling där utfall jämförs mot budget och prognos.

# Kommentar till 2022 års finansiella utveckling

## Intäkter, kostnader och resultat

FlexQube har haft sitt mest framgångsrika år hittills med rekordnivåer av både orderingång och försäljning. Robotprodukterna, har haft en stark tillväxt under året och företaget har gjort stora framsteg mot en fullskalig lansering av FlexQube Navigator AMR. Dessutom har företaget haft stor tillväxt som varumärke för traditionella mekaniska vagnar och uppnått nya rekordvolymer för Liftrunner-konceptet, vilket är ett utmärkt komplement till de mekaniska vagnarna.

Nettoomsättningen ökade med 81,7 procent till 204,6 MSEK (112,6). Rensat för valutakurspåverkan mellan jämförelseperioderna har nettoomsättningen ökat med 65,4 procent. Rörelseresultatet före avskrivningar (EBITDA) uppgick till -1,5 MSEK (-17,4) medan rörelseresultatet före finansiella poster (EBIT) uppgick till -6,4 MSEK (-22,0). Rörelsemarginalen har förbättrats mot föregående år vilket delvis beror på prishöjningar och en mer gynnsam produktmix.

Personalkostnaderna har ökat med 56,4 procent jämfört med föregående period på grund av flera nyanställningar både i Sverige och Nordamerika. Övriga externa kostnader har ökat med 61,5 procent, vilket inkluderar högre lokalkostnader för att anpassa sig till de större volymerna och en flytt till ett nytt huvudkontor i Sverige. Kostnaderna för marknadsföring har ökat med nästan 60 procent jämfört med föregående år på grund av att företaget har deltagit i fler mässor under 2022 och ökat sin marknadsaktivitet generellt efter pandemin. På grund av den högre orderingången och det ökade antalet anställda har kostnaderna för resor mer än fördubblats jämfört med föregående år.

Resultat före skatt uppgick till -7,0 MSEK (-22,5) och resultat per aktie uppgick till -0,9 SEK (-2,7).

## Kassaflöde och finansiell ställning

Periodens kassaflöde uppgick till -5,6 MSEK (21,5). Kassaflödet från den löpande verksamheten var -22,0 MSEK (-29,3), vilket huvudsakligen beror på det förbättrade resultatet. Vid balansdagen uppgick omsättningstillgångarna till 131,6 MSEK (112,3), varav varulager utgjorde 51,4 MSEK (33,4), kundfordringar utgjorde 43,6 MSEK (32,6) och likvida medel uppgick till 30,4 MSEK (34,9).

Kassaflödet från investeringsverksamheten uppgick till -8,1 MSEK (-2,8). Förändringen beror delvis på ökade investeringar i samband med byte av lokal i Sverige och produktutveckling av FlexQube Navigator AMR. Immateriella anläggningstillgångar uppgick till 17,5 MSEK (14,8), vilket främst bestod av utvecklingskostnader för FlexQubes robotsystem och andra utgifter för IT- och mjukvarulösningar, patent, varumärken och konceptuell utveckling av FlexQubes mekaniska byggblock.

Kassaflödet från finansieringsverksamheten uppgick till 24,5 MSEK (53,7). Förändringen beror främst på föregående års nyemission. Under perioden fortsatte vi fakturabelåning hos finansieringspartners, vilket redovisas som en finansiell skuld för den amerikanska verksamheten och påverkar denna del av kassaflödet positivt.

Eget kapital uppgick till 69,3 MSEK (72,1) vid periodens utgång, och koncernens soliditet uppgick till 45 procent (55 procent).

# Moderbolaget

FlexQube AB (publ) i Mölndal med org.nr. 556905-3944 är koncernens moderbolag. I samband med bolagets börsintroduktion har moderbolaget upprättat en förvaltningsfunktion för koncernen, inom ramen av företagsledning och styrning. Alla övriga verksamhetsrelaterade transaktioner som ej berör koncernförvaltning, med externa och/eller koncerninterna parter omsätts primärt av dotterbolagen.

# Väsentliga händelser under räkenskapsåret

## FlexQubes automationskoncept

Bolaget har under året fortsatt investera i automationskonceptet. FlexQubes AGV-system eQart® Line har visat hög tillväxt i både antal ordrar, genomsnittlig orderstorlek och vi har även utvecklat installationskonceptet ute hos våra kunder. Fler och fler eQart® Line levereras till kunder som får omedelbart värde, vilket minskar tidsåtgången på att flytta material internt och personalen kan fokusera på mer värdefulla uppgifter.

Vi har flera installationer igång på alla våra marknader, Europa, USA och Mexiko, med upp till åtta eQart® Line som körs samtidigt. Vi har lanserat flera mjukvaruuppdateringar under året, vilket gett kunderna fler funktioner OTA (over the air) för att förbättra det värde som skapas även efter initial uppstart. Under året genomfördes även en större uppgradering av eQart® Lines styrdator med kraftigare processor, bättre kamera och helt nya möjligheter till integration med andra system såsom rullbord och annan utrustning med sensorer.

Under året visade FlexQube upp sitt nya AMR-system, FlexQube Navigator AMR, på Modex-mässan i Atlanta. Produkten blev utsett till en av vinnarna i Robotics Innovation Award. FlexQube Navigator AMR vann utmärkelsen inom kategorin Teknologi, Produkt och Service Innovation där kriteriet var: "Ny kommersiell lösning som har potential att positivt påverka marknaden eller hela robotsektorn". Produkten kommer officiellt lanseras under 2023.

## Försäljning och marknadsföring

Bolaget visar upp ett nytt rekordår avseende försäljning med en nettoomsättning om 204,6 MSEK, vilket är en tillväxt med 82% jämfört med 2021. Målsättningar har varit höga och vi växer på samtliga marknader. Effekterna från coronapandemin ebbar ut vilket har möjliggjort betydligt fler kundbesök och vi har även deltagit på flera mässor under året. Bolaget fortsätter öka sina investeringar i både marknadsföring och utökad säljkår för att stötta de ökade volymerna.

## Organisation och kapacitet

FlexQube har fortsatt växa under 2022 och har utökat organisationen inom alla områden, både genom egna anställda men även ökade resurser hos våra externa samarbetspartners.

Sedan 2020 har bolaget montering i egen regi och vi märker tydligt av ökad effektivitet och kostnadsbesparingar från beslutet om insourcing av montering och distribution. Under hösten 2022 flyttade bolaget nya lokaler i Mölndal, utanför Göteborg. Här har vi nu samlokaliserat lager och monteringsverksamhet med vårt robotutvecklingsteam och övriga tjänster på huvudkontoret. Lokalerna är väl anpassade efter vår verksamhet och har redan genererat ytterligare positiv energi för hela organisationen.

Anders Fogelberg har, efter elva år som VD, i samförstånd med styrelsen, beslutat att lämna rollen som VD och i stället ytterligare stärka FlexQubes fortsatta expansion, med fokus på global strategi-, sälj- och affärsutveckling. Bolaget har rekryterat en ny VD, Mårten Frostne, som tillträder sin roll under sommaren 2023.

## Marknadsrisk

Koncernen utsätts för valutarisker vilka uppstår från olika valutaexponeringar men framför allt avseende euro (EUR), dollar (USD) och pund (GBP).

Valutarisken beror på att en del av koncernens intäkter är i EUR för den europeiska marknaden, medan rörelsekostnaderna i huvudsak är i SEK. Den amerikanska enheten har lokal tillverkning och supply chain i USA där endast begränsade inköp sker i annan valuta än USD. Därmed är valutarisken begränsad för den amerikanska enheten, undantaget eventuella koncerninterna transaktioner.

## Ränterisk

FlexQubes ränterisk uppstår främst genom långfristig upplåning samt tillgänglig checkkredit och fakturabelåning. FlexQube övervakar kontinuerligt ränteförändringar i marknaden.

## Kreditrisk

Som i alla verksamheter finns det en kreditrisk avseende kundernas betalningsförmåga, särskilt då försäljning sker på olika geografiska marknader. FlexQube försöker minimera denna risk genom att aktivt arbeta med att minska kredittiden och i vissa fall kräva förskottsbetalningar från kunder. Avsättning för osäkra fordringar baseras på historiska förluster och bedöms vara tillräcklig.

## Likviditetsrisk

FlexQubes likviditetsrisk är framför allt relaterad till att koncernens större kunder kräver långa betalningsperioder och att företaget befinner sig i en expansiv fas. För att hantera denna risk säkerställer FlexQube att tillräckligt med likvida medel finns tillgängliga genom en försiktig likviditetshantering, där ledningen följer löpande prognoser för koncernens likviditetsreserv inklusive outnyttjade kreditfaciliteter och förväntade kassaflöden.

FlexQube arbetar aktivt med att sänka risken genom befintliga globala finansieringsavtal som säkerställer ett tillfredsställande kassaflöde. Dessutom hanteras likviditetsrisken löpande i samarbete med koncernens långivare och andra finansiella samarbetspartners. På detta sätt säkerställer FlexQube att dess likviditetsrisk hanteras på ett effektivt sätt och att företaget har tillräckligt med likvida medel för att upprätthålla sin verksamhet och möta sina åtaganden, trots de utmaningar som kan uppstå på grund av dess affärsmodell och expansiva tillväxtfas.

## Konkurrens

FlexQube är ett internationellt företag som står inför risker relaterade till marknad och konkurrens. Konkurrensen på marknaden kan öka och påverka försäljningen negativt, vilket i sin tur kan minska företagets lönsamhet. FlexQube arbetar aktivt med att vara ledande inom sitt verksamhetsområde genom att satsa på produktutveckling och diversifiering av sitt produkterbjudande.

## Kompetens och nyckelpersoner

Betydande erfarenhet och kompetens finns inom FlexQube i egenskap av ledande befattningshavare och andra nyckelpersoner. FlexQube är beroende av att behålla och rekrytera personer med rätt kompetens för att fullfölja sina expansionsplaner och för att kunna fortsätta utveckla nya produkter. Om FlexQube saknar tillräckliga resurser kan det leda till att bolagets framtida expansion och tillväxtmål måste avbrytas eller reduceras i omfattning, vilket skulle kunna påverka företagets konkurrenskraft och lönsamhet på sikt.

## Covid-19

En risk för bolaget från 2020 är spridningen av coronaviruset. Det är svårt att överskåda potentiella konsekvenser av pandemin. Men pandemin kan både direkt och indirekt ha påtaglig effekt på bolagets verksamhet i form. Exempel kan vara produktionssvårigheter på grund av sjukfrånvaro, problem med komponentleveranser från externa leverantörer, minskad efterfrågan på koncernens produkter i händelse av konjunkturnedgång eller stängd verksamhet hos kunder. Det kan också innebära svårigheter att leda bolaget om ledande befattningshavare eller andra nyckelpersoner har längre sjukfrånvaro, kreditförluster.

# Väsentliga händelser efter räkenskapsårets utgång

FlexQube har ingått ett nytt kreditfacilitetsavtal med Danske Bank A/S och ALMI om totalt 45 MSEK. 5 MSEK avser långfristigt lån och 40 MSEK tillgängliggörs via en löpande checkkredit. De nya faciliteterna skall primärt användas för att finansiera ett löpande behov av ökat rörelsekapital vid lanseringen av bolagets nya automationsprodukt FlexQube Navigator AMR under 2023.

Vid extra bolagsstämma beslutades bland annat om nytt incitamentsprogram för kommande VD i bolaget. Bolagets kommande VD tecknar 110 000 teckningsoptioner vilket tillförde bolaget cirka 1,5 MSEK.

# Förslag till vinstdisposition

Mot bakgrund av att koncernen befinner sig i en expansions- och tillväxtfas föreslår bolagets styrelse att vinsten skall balanseras i ny räkning och att inte lämna någon utdelning till aktieägarna. Vad beträffar koncernens och moderbolagets resultat och ställning i övrigt, hänvisas till efterföljande balans- och resultaträkningar, förändring i eget kapital samt kassaflödesanalyser med tillhörande noter. Bolagsstyrningsrapport finns på sidorna 116-122.

## Till årsstämmans förfogande står följande medel i moderbolaget (TSEK):

- Balanserad vinst: 2 793
- Överkursfond: 144 979
- Årets resultat: 59
- **Totalt**: 147 831

Styrelsen föreslår att 147 831 TSEK balanseras i ny räkning.

# Aktien

FlexQubes aktie är listad på Nasdaq First North Stockholm sedan 14 december 2017 under beteckning FLEXQ. FlexQubes aktiekapital uppgick den 31 december 2022 till 0,8 MSEK fördelat på 8 233 333 utestående aktier med lika rätt. För mer information om aktien, se Aktiekapital, aktien och ägarförhållanden på sidorna 39-41.

# Finansiella Rapporter

## Koncernens balansräkning

### Tillgångar

| TSEK | Noter | 2022-12-31 | 2021-12-31 |
|------|-------|------------|------------|
| **TILLGÅNGAR** | | | |
| **Anläggningstillgångar** | | | |
| Immateriella anläggningstillgångar | 11 | | |
| Balanserade ut | | | |

# Koncernens balansräkning (fortsättning)

## Tillgångar (fortsättning)

| TSEK | Noter | 2022-12-31 | 2021-12-31 |
|------|-------|------------|------------|
| **Immateriella anläggningstillgångar** | | | |
| Balanserade utgifter för utvecklingsarbeten och liknande arbeten | | 14 664 | 11 985 |
| Koncessioner, patent, licenser, varumärken | | 2 790 | 2 562 |
| Övriga immateriella anläggningstillgångar | | - | 239 |
| **Summa immateriella anläggningstillgångar** | | **17 454** | **14 787** |
| | | | |
| **Materiella anläggningstillgångar** | 12 | | |
| Maskiner och andra tekniska anläggningar | | 2 301 | 1 890 |
| Inventarier, verktyg och installationer | | 2 335 | 1 066 |
| **Summa materiella anläggningstillgångar** | | **4 636** | **2 956** |
| | | | |
| **Summa anläggningstillgångar** | | **22 089** | **17 743** |
| | | | |
| **Omsättningstillgångar** | | | |
| **Varulager** | 15 | **51 430** | **33 407** |
| **Summa varulager m.m.** | | **51 430** | **33 407** |
| | | | |
| **Kortfristiga fordringar** | | | |
| Kundfordringar | | 43 601 | 32 634 |
| Övriga fordringar | | 1 641 | 5 247 |
| Förutbetalda kostnader och upplupna intäkter | 16 | 4 487 | 6 049 |
| **Summa kortfristiga fordringar** | | **49 729** | **43 931** |
| | | | |
| Kassa och bank | 24 | 30 452 | 34 925 |
| **Summa omsättningstillgångar** | | **131 612** | **112 262** |
| | | | |
| **SUMMA TILLGÅNGAR** | | **153 701** | **130 005** |

## Eget kapital och skulder

| TSEK | Noter | 2022-12-31 | 2021-12-31 |
|------|-------|------------|------------|
| **EGET KAPITAL** | | | |
| Aktiekapital | 17 | 823 | 823 |
| Övrigt tillskjutet kapital | | 148 232 | 147 589 |
| Balanserat resultat m.m. | | -72 655 | -53 737 |
| Årets resultat | | -7 068 | -22 559 |
| **Summa eget kapital** | | **69 332** | **72 116** |
| | | | |
| **Långfristiga skulder** | 18 | | |
| Skulder till kreditinstitut | 19 | 4 344 | 6 656 |
| Övriga långfristiga skulder | | 1 547 | 1 329 |
| **Summa långfristiga skulder** | | **5 892** | **7 985** |
| | | | |
| **Kortfristiga skulder** | | | |
| Leverantörsskulder | | 21 896 | 26 692 |
| Checkräkningskredit** | 20 | 2 308 | - |
| Skulder till kreditinstitut | 19 | 22 007 | 6 741 |
| Aktuella skatteskulder | | 5 | 5 |
| Övriga kortfristiga skulder | | 15 052 | 5 517 |
| Upplupna kostnader och förutbetalda intäkter | 21 | 17 208 | 10 950 |
| **Summa kortfristiga skulder** | | **78 477** | **49 904** |
| | | | |
| **SUMMA EGET KAPITAL OCH SKULDER** | | **153 701** | **130 005** |

**Specifikation av checkräkningslimit och outnyttjad del av check för respektive period anges nedan:

| Specifikation av checkräkningslimit och outnyttjad del av check: | 2022-12-31 | 2021-12-31 |
|------------------------------------------------------------------|------------|------------|
| Checkräkningskredit (TSEK) | 2 300 | 2 300 |
| Outnyttjad del av check (TSEK) | - | 2 300 |

# Koncernens resultaträkning

| TSEK | Noter | 2022 | 2021 |
|------|-------|------|------|
| Nettoomsättning | 2 | 204 594 | 112 630 |
| Aktiverat arbete | | 1 453 | 769 |
| Övriga rörelseintäkter* | 3 | 3 499 | 681 |
| **Summa rörelseintäkter** | | **209 547** | **114 080** |
| | | | |
| **RÖRELSENS KOSTNADER** | | | |
| Handelsvaror | | -98 096 | -60 362 |
| Övriga externa kostnader | 5 | -63 362 | -39 238 |
| Personalkostnader | 4 | -49 604 | -31 707 |
| Övriga rörelsekostnader* | 7 | - | -136 |
| **EBITDA** | | **-1 514** | **-17 362** |
| Avskrivningar av anläggningstillgångar | 6 | -4 851 | -4 633 |
| **Summa rörelsekostnader** | | **-215 912** | **-136 076** |
| **Rörelseresultat (EBIT)** | | **-6 365** | **-21 995** |
| | | | |
| **RESULTAT FRÅN FINANSIELLA POSTER** | | | |
| Övriga ränteintäkter och liknande resultatposter | 9 | 24 | - |
| Räntekostnader och liknande resultatposter | 9 | -656 | -538 |
| **Summa finansiella poster** | | **-632** | **-538** |
| | | | |
| **Resultat efter finansiella poster** | | **-6 997** | **-22 533** |
| Skatt på periodens resultat | 10 | -71 | -26 |
| **PERIODENS RESULTAT** | | **-7 068** | **-22 559** |
| | | | |
| **HÄNFÖRLIGT TILL:** | | | |
| Moderföretagets ägare | | -7 068 | -22 559 |
| Resultat per aktie hänförligt till moderföretagets ägare | | -0,9 | -2,7 |

*Innehåller valutakursförändringar av rörelseposter

# Koncernens förändringar i eget kapital

| TSEK | Aktiekapital | Övrigt tillskjutet kapital | Balanserat resultat m.m. | Totalt eget kapital |
|------|--------------|---------------------------|-------------------------|-------------------|
| **Ingående balans 2021-01-01** | 743 | 97 069 | -55 345 | 42 468 |
| Årets resultat | | | -22 559 | -22 559 |
| Valutakursdifferenser vid omräkning av utländska dotterföretag | | | 1 608 | 1 608 |
| Inbetald premie vid utfärdande av teckningsoption | | 439 | | 439 |
| Nyemission | 80 | 55 920 | | 56 000 |
| Emissionskostnader | | -5 839 | | -5 839 |
| **UTGÅENDE BALANS 2021-12-31** | **823** | **147 589** | **-76 296** | **72 116** |
| | | | | |
| **Ingående balans 2022-01-01** | 823 | 147 589 | -76 296 | 72 116 |
| Årets resultat | | | -7 068 | -7 068 |
| Valutakursdifferenser vid omräkning av utländska dotterföretag | | | 3 641 | 3 641 |
| Inbetald premie vid utfärdande av teckningsoption | | 643 | | 643 |
| **UTGÅENDE BALANS 2022-12-31** | **823** | **148 232** | **-79 723** | **69 332** |

# Koncernens kassaflödesanalys

| TSEK | Noter | 2022 | 2021 |
|------|-------|------|------|
| **DEN LÖPANDE VERKSAMHETEN** | | | |
| Rörelseresultat före finansiella poster | | -6 365 | -21 995 |
| **Justeringar för poster som ej ingår i kassaflödet** | | | |
| Avskrivningar | | 4 851 | 4 633 |
| Övriga poster som inte ingår i kassaflödet | 23 | 5 664 | 1 744 |
| Erhållen ränta | | 24 | - |
| Erlagd ränta | | -656 | -538 |
| Betald inkomstskatt | | -70 | -25 |
| **Kassaflöde från den löpande verksamheten före ändringar av rörelsekapital** | | **3 448** | **-16 182** |
| | | | |
| **Kassaflöde från förändringar i rörelsekapital** | | | |
| Förändringar av varulager | | -21 831 | -14 584 |
| Förändringar av rörelsefordringar | | -1 058 | -19 561 |
| Förändringar av rörelseskulder | | -2 523 | 21 003 |
| **Kassaflöde från den löpande verksamheten** | | **-21 964** | **-29 323** |
| | | | |
| **INVESTERINGSVERKSAMHETEN** | | | |
| Förvärv av immateriella anläggningstillgångar | | -6 012 | -2 089 |
| Förvärv av materiella anläggningstillgångar | | -2 075 | -745 |
| **Kassaflöde från investeringsverksamheten** | | **-8 087** | **-2 834** |
| | | | |
| **FINANSIERINGSVERKSAMHETEN** | | | |
| Nyemission | | - | 56 000 |
| Emissionskostnader | | - | -5 839 |
| Teckningsoptionsprogram | | 643 | 439 |
| Förändring kortfristiga finansiella skulder | | 17 573 | 4 430 |
| Upptagna lån | | 9 000 | 11 500 |
| Amortering av lån | | -2 311 | -12 533 |
| Amortering av finansiell leasingskuld | | -454 | -319 |
| **Kassaflöde från finansieringsverksamheten** | | **24 451** | **53 678** |
| | | | |
| **PERIODENS KASSAFLÖDE** | | **-5 601** | **21 520** |
| Likvida medel vid periodens början | | 34 925 | 13 389 |
| Kursdifferens i likvida medel | | 1 128 | 15 |
| **LIKVIDA MEDEL VID PERIODENS UTGÅNG** | | **30 452** | **34 925** |

# Moderbolagets resultaträkning

| TSEK | Noter | 2022 | 2021 |
|------|-------|------|------|
| Nettoomsättning | 2 | - | 2 419 |
| Övriga rörelseintäkter | 3 | 3 865 | 1 797 |
| **Summa rörelseintäkter** | | **3 865** | **4 216** |
| | | | |
| **RÖRELSENS KOSTNADER** | | | |
| Övriga externa kostnader | 5 | -2 211 | -1 724 |
| Personalkostnader | 4 | -685 | -695 |
| Övriga rörelsekostnader | 7 | - | - |
| **Summa rörelsekostnader** | | **-2 896** | **-2 419** |
| **EBITDA** | | **969** | **1 797** |
| **Rörelseresultat (EBIT)** | | **969** | **1 797** |
| | | | |
| **RESULTAT FRÅN FINANSIELLA POSTER** | | | |
| Övriga ränteintäkter och liknande resultatposter | 9 | 2 505 | 1 389 |
| Räntekostnader och liknande resultatposter | 9 | -15 | -77 |
| **Summa finansiella poster** | | **2 490** | **1 312** |
| | | | |
| **Resultat efter finansiella poster** | | **3 459** | **3 109** |
| Bokslutsdispositioner | 8 | -3 400 | -1 797 |
| Skatt på periodens resultat | 10 | - | - |
| **PERIODENS RESULTAT** | | **59** | **1 312** |

# Moderbolagets förändringar i eget kapital

| TSEK | Aktiekapital | Överkursfond | Balanserat resultat m.m. | Totalt eget kapital |
|------|--------------|--------------|-------------------------|-------------------|
| **Ingående balans 2021-01-01** | 743 | 93 816 | 1 481 | 96 041 |
| Årets resultat | | | 1 312 | 1 312 |
| Inbetald premie vid utfärdande av teckningsoption | | 439 | | 439 |
| Nyemission | 80 | 55 920 | | 56 000 |
| Emissionskostnader | | -5 839 | | -5 839 |
| **UTGÅENDE BALANS 2021-12-31** | **823** | **144 336** | **2 794** | **147 953** |
| | | | | |
| **Ingående balans 2022-01-01** | 823 | 144 336 | 2 794 | 147 953 |
| Årets resultat | | | 59 | 59 |
| Inbetald premie vid utfärdande av teckningsoption | | 643 | | 643 |
| **UTGÅENDE BALANS 2022-12-31** | **823** | **144 979** | **2 853** | **148 655** |

# Moderbolagets balansräkning

| TSEK | Noter | 2022-12-31 | 2021-12-31 |
|------|-------|------------|------------|
| **TILLGÅNGAR** | | | |
| **Anläggningstillgångar** | | | |
| **Finansiella anläggningstillgångar** | | | |
| Andelar i koncernföretag | 14 | 85 570 | 76 405 |
| Fordringar hos koncernföretag | 13 | 65 583 | 40 389 |
| **Summa finansiella anläggningstillgångar** | | **151 153** | **116 794** |
| **Summa anläggningstillgångar** | | **151 153** | **116 794** |
| | | | |
| **Omsättningstillgångar** | | | |
| **Kortfristiga fordringar** | | | |
| Fordringar hos koncernföretag | | 2 155 | 4 266 |
| Övriga fordringar | | 100 | - |
| Förutbetalda kostnader och upplupna intäkter | 16 | 59 | 50 |
| **Summa kortfristiga fordringar** | | **2 314** | **4 316** |
| | | | |
| Kassa och bank | 24 | 9 172 | 30 119 |
| **Summa omsättningstillgångar** | | **11 486** | **34 435** |
| **SUMMA TILLGÅNGAR** | | **162 639** | **151 229** |

## EGET KAPITAL OCH SKULDER

| TSEK | Noter | 2022-12-31 | 2021-12-31 |
|------|-------|------------|------------|
| **EGET KAPITAL** | | | |
| Aktiekapital | 17 | 823 | 823 |
| **Summa bundet eget kapital** | | **823** | **823** |
| | | | |
| Överkursfond | | 144 979 | 144 336 |
| Balanserat resultat | | 2 793 | 1 481 |
| Årets resultat | | 59 | 1 312 |
| **Summa fritt eget kapital** | | **147 831** | **147 130** |
| **Summa eget kapital** | | **148 655** | **147 953** |
| | | | |
| **Kortfristiga skulder** | | | |
| Leverantörsskulder | | 1 | - |
| Skulder till koncernföretag | | 3 400 | 1 958 |
| Övriga kortfristiga skulder | | 9 978 | 572 |
| Upplupna kostnader och förutbetalda intäkter | 21 | 605 | 746 |
| **Summa kortfristiga skulder** | | **13 984** | **3 276** |
| **SUMMA EGET KAPITAL OCH SKULDER** | | **162 639** | **151 229** |

# Moderbolagets kassaflödesanalys

| TSEK | Noter | 2022 | 2021 |
|------|-------|------|------|
| **DEN LÖPANDE VERKSAMHETEN** | | | |
| Rörelseresultat före finansiella poster | | 969 | 1 797 |
| **Justeringar för poster som ej ingår i kassaflödet** | | | |
| Övriga poster som inte ingår i kassaflödet | 23 | -1 699 | -2 622 |
| Erhållen ränta | | 2 505 | 1 389 |
| Erlagd ränta | | -15 | -77 |
| **Kassaflöde från den löpande verksamheten före förändringar av rörelsekapital** | | **1 760** | **487** |
| | | | |
| **Kassaflöde från förändringar i rörelsekapital** | | | |
| Förändringar av rörelsefordringar | | 2 002 | -3 472 |
| Förändringar av rörelseskulder | | 105 | 334 |
| **Kassaflöde från den löpande verksamheten** | | **3 867** | **-2 651** |
| | | | |
| **INVESTERINGSVERKSAMHETEN** | | | |
| Lämnat kapitaltillskott | | -9 165 | -19 097 |
| Lämnade lån till koncernföretag | | -69 972 | -45 900 |
| Erhållna amorteringar från koncernföretag | | 46 478 | 39 493 |
| **Kassaflöde från investeringsverksamheten** | | **-32 660** | **-25 504** |
| | | | |
| **FINANSIERINGSVERKSAMHETEN** | | | |
| Nyemission | | - | 56 000 |
| Emissionskostnader | | - | -5 839 |
| Teckningsoptionsprogram | | 643 | 439 |
| Upptagna lån | | 9 000 | 11 500 |
| Amortering av lån | | - | -11 500 |
| Lämnat koncernbidrag | | -1 797 | - |
| **Kassaflöde från finansieringsverksamheten** | | **7 846** | **50 600** |
| | | | |
| **PERIODENS KASSAFLÖDE** | | **-20 947** | **22 445** |
| Likvida medel vid periodens början | | 30 119 | 7 674 |
| **LIKVIDA MEDEL VID PERIODENS UTGÅNG** | | **9 172** | **30 119** |

# NOTER

## Gemensamma för koncern och moderbolag

### Not 1 REDOVISNINGS- OCH VÄRDERINGSPRINCIPER

Aktuell finansiell information är upprättad enligt ÅRL och Bokföringsnämndens allmänna råd BFNAR 2012:1 Årsredovisning och koncernredovisning (K3). Redovisningsprinciperna är oförändrade jämfört med föregående år.

#### KONCERNREDOVISNING

Företag där FlexQube innehar majoriteten av rösterna på bolagsstämman klassificeras som dotterföretag och konsolideras i koncernredovisningen. Dotterföretagen inkluderas i koncernredovisningen från och med den dag då det bestämmande inflytandet överförs till koncernen. De exkluderas ur koncernredovisningen från och med den dag då det bestämmande inflytandet upphör. Koncernens bokslut är upprättat enligt förvärvsmetoden. Förvärvstidpunkten är den tidpunkt då det bestämmande inflytandet erhålls. Identifierbara tillgångar och skulder värderas inledningsvis till verkliga värden vid förvärvstidpunkten. Goodwill/Negativ Goodwill utgörs av mellanskillnaden mellan de förvärvade identifierbara nettotillgångarna vid förvärvstillfället och anskaffningsvärdet inklusive värdet av minoritetsintresset, och värderas initialt till anskaffningsvärdet. Koncernen har aldrig redovisat någon Goodwill.

Mellanhavanden mellan koncernföretag elimineras i sin helhet.

Dotterföretag i andra länder upprättar sin årsredovisning i utländsk valuta. Vid konsolideringen omräknas posterna i dessa dotterföretags balans- och resultaträkningar till balansdagskurs respektive avistakurs för den dag respektive affärshändelse ägde rum. De valutakursdifferenser som uppkommer vid omräkning av balansräkning för utländska dotterbolag redovisas i ackumulerade valutakursdifferenser i koncernens eget kapital.

#### Utländska valutor

Monetära tillgångs- och skuldposter i utländsk valuta värderas till balansdagens avistakurs. Transaktioner i utländsk valuta omräknas enligt transaktionsdagens avistakurs.

#### Intäkter

Försäljning av varor redovisas när väsentliga risker och fördelar övergår från säljare till köpare i enlighet med försäljningsvillkoren. Försäljningen redovisas efter avdrag för moms och rabatter. Försäljning av tjänster redovisas när tjänsten i fråga har blivit utförd enligt avtalade villkor.

#### Leasingavtal

Leasingavtal som innebär att de ekonomiska riskerna och fördelarna med att äga en tillgång i allt väsentligt överförs från leasegivaren till leasetagaren klassificeras i koncernredovisningen som finansiella leasingavtal. Finansiella leasingavtal medför att rättigheter och skyldigheter redovisas som tillgång respektive skuld i balansräkningen. Tillgången och skulden värderas initialt till det lägsta av tillgångens verkliga värde och nuvärdet av minimileaseavgifterna. Utgifter som direkt kan hänföras till leasingavtalet läggs till tillgångens värde. Leasingavgifterna fördelas på ränta och amortering enligt effektivräntemetoden. Variabla avgifter redovisas som kostnad i den period de uppkommer. Den leasade tillgången skrivs av linjärt över den bedömda nyttjandeperioden. Leasingavtal där de ekonomiska fördelar och risker som är hänförliga till leasingobjektet i allt väsentligt kvarstår hos leasegivaren, klassificeras som operationell leasing. Betalningar, inklusive en första förhöjd hyra, enligt dessa avtal redovisas som kostnad linjärt över leasingperioden.

#### Ersättningar till anställda (kortfristiga ersättningar)

Kortfristiga ersättningar i koncernen utgörs av lön, sociala avgifter, betald semester, betald sjukfrånvaro, sjukvård och bonus. Kortfristiga ersättningar redovisas som en kostnad och en skuld då det finns en legal eller informell förpliktelse att betala ut en ersättning.

#### Ersättningar efter avslutad anställning

Koncernen innehar endast avgiftsbestämda planer. I avgiftsbestämda planer betalar företaget fastställda avgifter till ett annat företag och har inte någon legal eller informell förpliktelse att betala något ytterligare även om det andra företaget inte kan uppfylla sitt åtagande. Koncernens resultat belastas för kostnader i takt med att de anställdas pensionsberättigade tjänster utförts.

#### Inkomstskatter

Aktuella skatter värderas utifrån de skattesatser och skatteregler som gäller på balansdagen. Uppskjutna skatter värderas utifrån de skattesatser och skatteregler som är beslutade före balansdagen.

#### Immateriella tillgångar

Immateriella anläggningstillgångar redovisas till anskaffningsvärde minskat med ackumulerade avskrivningar och nedskrivningar. I koncernredovisningen tillämpas aktiveringsmodellen för internt upparbetade immateriella tillgångar vilket innebär att:

Utvecklingskostnader som är direkt hänförliga till utveckling och testning av identifierbara och unika programvaruprodukter som kontrolleras av koncernen, redovisas som immateriella tillgångar när följande kriterier är uppfyllda:

• Det är tekniskt möjligt att färdigställa programvaran så att den kan användas,
• Företagets avsikt är att färdigställa programvaran och att använda eller sälja den,
• Det finns förutsättningar att använda eller sälja programvaran,
• Det kan visas hur programvaran genererar troliga framtida ekonomiska fördelar,
• Adekvata tekniska, ekonomiska och andra resurser för att fullfölja utvecklingen och för att använda eller sälja programvaran finns tillgängliga, och
• De utgifter som är hänförliga till programvaran under dess utveckling kan beräknas på ett tillförlitligt sätt.

Direkt hänförbara utgifter som balanseras som en del av utvecklingsprogram, innefattar utgifter för anställda och en skälig andel av indirekta kostnader. Balanserade utvecklingskostnader redovisas som immateriella tillgångar och skrivs av från den tidpunkt då tillgången är färdig att användas. Avskrivningar görs linjärt över den bedömda nyttjandetiden. Avskrivningstiden för internt upparbetade immateriella

## NYCKELTALSDEFINITIONER

**Balanslikviditet:** Omsättningstillgångar i procent av kortfristiga skulder.

**EBIT:** Rörelseresultat före räntor och skatt.

**EBITDA:** Rörelseresultat före avskrivningar, räntor och skatt.

**Eget kapital per aktie:** Eget kapital vid periodens slut dividerat med justerat antal aktier i slutet av räkenskapsperioden.

**Försäljningstillväxt:** Skillnaden i nettoomsättning mellan två perioder, dividerat med nettoomsättningen under den första perioden.

**Kassalikviditet:** Omsättningstillgångar exklusive varulager och inklusive outnyttjad del av checkräkningskredit i procent av kortfristiga skulder.

**Nettoskuld:** Bruttoskuld, det vill säga total lång- och kortfristig upplåning inklusive nyttjad del av checkräkningskredit, minus likvida medel, kortfristiga fordringar samt lätt realiserade tillgångar.

**Orderingång:** Värde av erhållna beställningar under angiven period.

**Resultat per aktie:** Periodens resultat i relation till justerat genomsnittligt antal aktier under räkenskapsperioden.

**Rörelsekapital:** Varulager samt kundfordringar minus leverantörsskulder.

**Rörelsemarginal:** Rörelseresultat efter avskrivningar i procent av nettoomsättning.

**Soliditet:** Relationen mellan bolagets egna kapital och de totala tillgångarna i bolagets balansräkning.

**Vinstmarginal:** Resultat före skatt i procent av nettoomsättning.

### Nyckeltalsdefinitioner av alternativa nyckeltal

**Kassalikviditet inklusive outnyttjad del av checkräkningskredit:** Omsättningstillgångar exklusive lager plus outnyttjad del av checkräkningskredit i procent av kortfristiga skulder.

**Rörelsekapital i procent av nettoomsättningen:** Totala omsättningstillgångar minus kortfristiga skulder i förhållande till nettoomsättning i procent.

## MODERFÖRETAGETS REDOVISNINGS- OCH VÄRDERINGSPRINCIPER

Samma redovisnings- och värderingsprinciper tillämpas i moderföretaget som i koncernen, förutom i de fall som anges nedan:

### Aktier och andelar i dotterföretag
Aktier och andelar i dotterföretag redovisas till anskaffningsvärde efter avdrag för eventuella nedskrivningar. I anskaffningsvärdet ingår köpeskillingen som erlagts för aktierna samt förvärvskostnader. Eventuella kapitaltillskott läggs till anskaffningsvärdet när de lämnas. Utdelning från dotterföretag redovisas som intäkt.

### Koncernbidrag
Koncernbidrag lämnade från moderföretag till dotterföretag och koncernbidrag erhållna till moderföretag från dotterföretag redovisas som bokslutsdisposition.

### Eget kapital
Eget kapital delas in i bundet och fritt kapital, i enlighet med årsredovisningslagens indelning.

### Nedskrivningsprövning av finansiella anläggningstillgångar
Vid varje balansdag bedömer bolaget om det finns någon indikation på nedskrivningsbehov i någon av de finansiella anläggningstillgångarna. Nedskrivning sker om värdenedgången bedöms vara bestående. Nedskrivning redovisas i resultaträkningsposten Resultat från övriga värdepapper och fordringar som är anläggningstillgångar. Nedskrivningsbehovet prövas individuellt för aktier och andelar och övriga enskilda finansiella anläggningstillgångar som är väsentliga.

### Kvittning av finansiell fordran och finansiell skuld
En finansiell tillgång och en finansiell skuld kvittas och redovisas med ett nettobelopp i balansräkningen endast då legal kvittningsrätt föreligger samt då en reglering med ett nettobelopp avses ske eller då en samtid avyttring av tillgången och reglering av skulden avses ske.

## JÄMFÖRELSESTÖRANDE POSTER

Jämförelsestörande poster redogörs för separat där nödvändigt för att förklara koncernens resultat. Med jämförelsestörande poster avses substantiella intäkts- eller kostnadsposter som bör särskilt uppmärksammas på grund av betydelsen av deras karaktär eller belopp.

## BEDÖMNINGAR OCH UPPSKATTNINGAR

Koncernen gör uppskattningar och antaganden om framtiden för att upprätta redovisningen enligt god redovisningssed. Uppskattningar och bedömningar är baserade på historiska erfarenheter och andra faktorer som utvärderas löpande. Faktiskt utfall kan skilja sig från gjorda antaganden och uppskattningar.

### Immateriella tillgångar
Det redovisade värdet är beroende av den framtida marknaden för produkterna och förväntade kassaflöden. Koncernen undersöker varje år om det föreligger något nedskrivningsbehov och bedömningen per den 31 december 2022 är att det redovisade värdet inte överstiger verkligt värde.

### Varulager
Lagervärdet är beroende av bedömningar avseende beräkningen av lagrets nettoförsäljningsvärde. Dessa bedömningar kan föranleda nedskrivningar av lagervärdet. Risken för att nettoförsäljningsvärdet är lägre än anskaffningsvärdet bedöms som låg, eftersom majoriteten av produkterna är okänsliga för trender och har lång hållbarhet.

## NOTER

### Not 2 NETTOOMSÄTTNINGENS FÖRDELNING

| NETTOOMSÄTTNINGENS FÖRDELNING, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Sverige | 8 543 | 3 426 |
| Europa | 47 670 | 32 216 |
| Nordamerika | 113 033 | 58 018 |
| U.K | 6 738 | 2 054 |
| Övriga världen | 28 610 | 16 916 |
| **Summa nettoomsättningens fördelning, koncernen** | **204 594** | **112 630** |

| NETTOOMSÄTTNINGENS FÖRDELNING, MODERBOLAGET | 2022 | 2021 |
|---|---|---|
| Sverige | - | 2 419 |
| Övriga världen | - | - |
| **Summa nettoomsättningens fördelning, moderbolag** | **-** | **2 419** |

### Not 3 ÖVRIGA RÖRELSEINTÄKTER

| ÖVRIGA RÖRELSEINTÄKTER, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Övriga intäkter | 263 | 140 |
| Erhållna bidrag* | 163 | 57 |
| Kursvinst rörelsen (netto) | 3 073 | 483 |
| **Summa övriga rörelseintäkter, koncernen** | **3 499** | **680** |

| ÖVRIGA RÖRELSEINTÄKTER, MODERBOLAG | 2022 | 2021 |
|---|---|---|
| Kursvinst på fordringar hos koncernföretag | 3 865 | 1 797 |
| **Summa övriga rörelseintäkter, moderbolag** | **3 865** | **1 797** |

### Not 4 PERSONAL

| MEDELANTAL ANSTÄLLDA I KONCERNEN | 2022 | 2021 |
|---|---|---|
| Medelantalet anställda har varit följande: | 60 | 39 |
| -Varav kvinnor | 15 | 9 |
| Inhyrd personal | 12 | 30 |

| KÖNSFÖRDELNING I STYRELSE OCH FÖRETAGSLEDNING PER BALANSDAG | 2022 | 2021 |
|---|---|---|
| Antal styrelseledamöter | 4 | 4 |
| -Varav kvinnor | - | - |
| Antal övriga ledande befattningshavare inkl. VD | 8 | 8 |
| -Varav kvinnor | 2 | 1 |

| LÖNER, ERSÄTTNINGAR M.M. TILL STYRELSEN, MODERBOLAG | 2022 | 2021 |
|---|---|---|
| Christian Thiel, styrelseordförande | 242 | 195 |
| Anders Ströby | 145 | 135 |
| Ulf Ivarsson | - | 75 |
| Mikael Bluhme | 145 | 135 |
| Per Augustsson | - | - |
| **Summa löner, ersättningar m.m. till styrelsen** | **532** | **540** |
| Totala sociala avgifter | 138 | 141 |

Styrelsen har ej erhållit någon rörlig ersättning, pension eller andra förmåner.

| LÖNER, ERSÄTTNINGAR M.M. TILL ANSTÄLLDA, DOTTERBOLAG | 2022 | 2021 |
|---|---|---|
| Anders Fogelberg (VD)* | 1 579 | 1 226 |
| Övriga ledande befattningshavare (7) | 6 843 | 4 882 |
| Övriga anställda | 30 612 | 19 235 |
| **Summa löner, ersättningar m.m. till anställda** | **39 034** | **25 343** |
| Totala sociala avgifter | 6 554 | 3 842 |

*Varav förmåner har utgått till VD med 96 TSEK (92 TSEK)

| PENSIONSKOSTNADER, DOTTERBOLAG* | 2022 | 2021 |
|---|---|---|
| Anders Fogelberg (VD) | 378 | 113 |
| Övriga ledande befattningshavare (7) | 1 050 | 460 |
| Övriga anställda | 774 | 640 |
| **Summa pensioner till anställda** | **2 202** | **1 213** |
| Varav särskild löneskatt på pensionskostnader | 347 | 220 |

*Pensionskostnader utgörs av under året betalda avtalspremier enligt avgiftsbestämda pensionsplaner.

Ingen pension utgick från moderbolaget under åren 2022 och 2021.

### Not 5 ERSÄTTNING TILL REVISORERNA

| ERSÄTTNING TILL REVISORER, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Revision, PwC | 350 | 250 |
| Andra uppdrag, PwC | 70 | 58 |
| Skatterådgivning, PwC | - | - |
| **Summa ersättning till revisorer, koncernen** | **420** | **308** |

| ERSÄTTNING TILL REVISORER, MODERBOLAG | 2022 | 2021 |
|---|---|---|
| Revisionsuppdrag, PwC | 350 | 250 |
| Andra uppdrag, PwC | 70 | 58 |
| Skatterådgivning, PwC | - | - |
| **Summa ersättning till revisorer, moderbolag** | **420** | **308** |

### Not 6 AVSKRIVNINGAR

Avskrivningar av immateriella anläggningstillgångar uppgår i koncernen till 3,3 MSEK (3,6). Avskrivningar av materiella anläggningstillgångar uppgår i koncernen till 1,5 MSEK (1). Inga avskrivningar har genomförts i moderbolaget.

### Not 7 ÖVRIGA RÖRELSEKOSTNADER

| ÖVRIGA RÖRELSEKOSTNADER, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Kursförlust rörelsen (netto) | - | 136 |
| **Summa övriga rörelsekostnader, koncernen** | **-** | **136** |

| ÖVRIGA RÖRELSEKOSTNADER, MODERBOLAGET | 2022 | 2021 |
|---|---|---|
| Kursförlust rörelsen (netto) | - | - |
| **Summa övriga rörelsekostnader, moderbolaget** | **-** | **-** |

### Not 8 BOKSLUTSDISPOSITIONER

| BOKSLUTSDISPOSITIONER, MODERBOLAG | 2022 | 2021 |
|---|---|---|
| Lämnade koncernbidrag | 3 400 | 1 797 |
| **Summa bokslutsdispositioner, moderbolag** | **3 400** | **1 797** |

### Not 9 RÄNTEINTÄKTER OCH RÄNTEKOSTNADER

| RÄNTEINTÄKTER OCH RÄNTEKOSTNADER, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Ränteintäkter | 24 | - |
| **Summa ränteintäkter, koncernen** | **24** | **-** |
| Räntekostnader | 656 | 538 |
| **Summa räntekostnader, koncernen** | **656** | **538** |

| RÄNTEINTÄKTER OCH RÄNTEKOSTNADER, MODERBOLAGET | 2022 | 2021 |
|---|---|---|
| Ränteintäkter | 2 505 | 1 389 |
| **Summa ränteintäkter, moderbolaget** | **2 505** | **1 389** |
| Varav koncerninterna ränteintäkter | 2 505 | 1 389 |
| Räntekostnader | 15 | 77 |
| **Summa räntekostnader, moderbolaget** | **15** | **77** |
| Varav koncerninterna räntekostnader | - | - |

### Not 10 INKOMSTSKATT

| INKOMSTSKATT, KONCERNEN | 2022 | 2021 |
|---|---|---|
| Aktuell skatt | -71 | -26 |
| Uppskjuten skatt | - | - |
| **Skatt på årets resultat, koncernen** | **-71** | **-26** |

| | 2022 | 2021 |
|---|---|---|
| Redovisat resultat före skatt | -6 997 | -22 533 |
| Skatt beräknad enligt gällande skattesats | 1 441 | 4 642 |
| Effekt av ej avdragsgilla kostnader | -76 | -15 |
| Under året uttnyttjande av tidigare års underskottsavdrag vars skattevärde ej redovisats som tillgång | 888 | 272 |
| Ökning av underskottsavdrag utan motsvarande aktivering av uppskjuten skatt | -2 254 | -4 899 |
| Effekt av utländska skattesatser | -71 | -26 |
| **Redovisad skattekostnad** | **-71** | **-26** |

Uppskjuten skattefordran på underskottsavdrag år 2022 eller år 2021 har ej redovisats, underskottet per balansdagen uppgår till 83,3 MSEK (76,3) i koncernen.

| INKOMSTSKATT, MODERBOLAGET | 2022 | 2021 |
|---|---|---|
| Aktuell skatt | - | - |
| Uppskjuten skatt | - | - |
| **Skatt på årets resultat, moderbolaget** | **-** | **-** |

| | 2022 | 2021 |
|---|---|---|
| Redovisat resultat före skatt | 59 | 1 312 |
| Skatt beräknad enligt gällande skattesats | -12 | -270 |
| Effekt av ej avdragsgilla kostnader | -3 | -2 |
| Under året uttnyttjande av tidigare års underskottsavdrag vars skattevärde ej redovisats som tillgång | 15 | 272 |
| **Redovisad skattekostnad** | **-** | **-** |

### Not 11 IMMATERIELLA TILLGÅNGAR, KONCERNEN

| BALANSERADE UTGIFTER FÖR UTVECKLINGSARBETEN | 2022 | 2021 |
|---|---|---|
| Ingående anskaffningsvärden | 17 207 | 15 290 |
| Årets anskaffningsvärde | 5 343 | 1 917 |
| Utrangeringar | -11 | - |
| **Utgående ack. anskaffningsvärden** | **22 540** | **17 207** |
| Ingående avskrivningar | -5 223 | -2 657 |
| Årets avskrivningar | -2 664 | -2 566 |
| Utrangeringar | 11 | - |
| **Utgående ack. avskrivningar** | **-7 876** | **-5 223** |
| **Utgående bokfört värde** | **14 664** | **11 985** |

| ÖVRIGA IMMATERIELLA RÄTTIGHETER | 2022 | 2021 |
|---|---|---|
| Ingående anskaffningsvärden | 2 268 | 2 268 |
| Utrangeringar | -2 268 | - |
| **Utgående ack. anskaffningsvärden** | **-** | **2 268** |
| Ingående avskrivningar | -2 028 | -1 398 |

### Not 12 MATERIELLA ANLÄGGNINGSTILLGÅNGAR, KONCERNEN

| MASKINER OCH ANDRA TEKNISKA ANLÄGGNINGAR | 2022 | 2021 |
|---|---|---|
| Ingående anskaffningsvärden | 2 426 | 1 702 |
| Årets anskaffningsvärde | 548 | 642 |
| Utrangeringar | -109 | - |
| Omräkningsdifferens | 17 | 82 |
| **Utgående ack. anskaffningsvärden** | **2 882** | **2 426** |
| Ingående avskrivningar | -536 | -313 |
| Årets avskrivningar | -597 | -223 |
| Utrangeringar | 109 | - |
| Omräkningsdifferens | 443 | - |
| **Utgående ack.avskrivningar** | **-581** | **-536** |
| **Utgående bokfört värde** | **2 301** | **1 890** |

2,1 (1,6) MSEK av posten hänför sig till anläggningstillgångar avseende tjänstebilar som har tagits upp i koncernens balansräkning som följd av finansiell leasing.

| INVENTARIER, VERKTYG OCH INSTALLATIONER | 2022 | 2021 |
|---|---|---|
| Ingående anskaffningsvärden | 3 274 | 2 467 |
| Årets anskaffningsvärde | 2 075 | 745 |
| Utrangeringar | -1 683 | - |
| Omräkningsdifferens | 101 | 61 |
| **Utgående ack. anskaffningsvärden** | **3 767** | **3 274** |
| Ingående avskrivningar | -2 208 | -1 369 |
| Årets avskrivningar | -909 | -839 |
| Utrangeringar | 1 683 | - |
| Omräkningsdifferens | 1 | - |
| **Utgående ack.avskrivningar** | **-1 433** | **-2 208** |
| **Utgående bokfört värde** | **2 335** | **1 066** |

### Not 13 FORDRINGAR HOS KONCERNFÖRETAG

| FORDRINGAR HOS KONCERNFÖRETAG, MODERBOLAG | 2022 | 2021 |
|---|---|---|
| Vid årets början | 40 389 | 31 360 |
| Tillkommande fordringar | 69 972 | 45 900 |
| Årets amorteringar | -37 312 | -20 374 |
| Aktieägartillskott | -9 165 | -19 097 |
| Valutaeffekter | 1 699 | 2 600 |
| **Summa fordringar hos koncernföretag, moderbolag** | **65 583** | **40 389** |

## Not 11 IMMATERIELLA TILLGÅNGAR, KONCERNEN (fortsättning)

| | 2022 | 2021 |
|---|---|---|
| Årets avskrivningar | -258 | -630 |
| Utrangeringar | 2 268 | - |
| Omklassificeringar | 18 | - |
| Utgående ack. avskrivningar | - | -2 028 |
| Utgående bokfört värde | - | 239 |

### KONCESSIONER, PATENT, LICENSER OCH VARUMÄRKEN

| | 2022 | 2021 |
|---|---|---|
| Ingående anskaffningsvärden | 3 995 | 3 824 |
| Årets anskaffningsvärde | 669 | 172 |
| Utgående ack. anskaffningsvärden | 4 664 | 3 995 |
| Ingående avskrivningar | -1 433 | -1 057 |
| Årets avskrivningar | -424 | -376 |
| Omklassificeringar | -18 | - |
| Utgående ack. avskrivningar | -1 875 | -1 433 |
| Utgående bokfört värde | 2 790 | 2 562 |

## Not 14 ANDELAR I KONCERNBOLAG

FlexQube AB (publ) med org.nr. 556905-3944, är moderbolag i FlexQube-koncernen. Av nedan tabell redovisas samtliga helägda dotterföretag med uppgift om firma, organisationsnummer, säte samt bokfört värde per den 31 december 2022.

| Koncernföretag | Org.nr. | Säte | Antal andelar | Bokfört värde 2022 | Bokfört värde 2021 | Förändring |
|---|---|---|---|---|---|---|
| FlexQube Europe AB | 556823-6078 | Mölndal, Sverige | 50 000 | 38 973 | 38 973 | - |
| FlexQube GmbH | HRB 110829 | Frankfurt am Main, Tyskland | 25 000 | 8 485 | 8 485 | - |
| FlexQube Inc | 90-0998273 | Delaware, USA | 1 000 | 33 838 | 25 290 | 8 548 |
| FlexQube Ltd | 11917032 | London, Storbritannien | 1 000 | 3 989 | 3 371 | 618 |
| FQ IP AB | 556905-4017 | Mölndal, Sverige | 50 000 | 285 | 285 | - |
| **Summa** | | | **127 000** | **85 570** | **76 405** | **9 165** |

Årets förändring av andelar i koncernföretag avser lämnat aktieägartillskott på 9 165 TSEK.

## Not 15 VARULAGER, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Produkter i arbete | 1 111 | 2 371 |
| Färdiga varor | 1 599 | 411 |
| Handelsvaror | 47 856 | 26 564 |
| Varor på väg | 864 | 4 060 |
| **Summa varulager, koncernen** | **51 430** | **33 407** |

## Not 16 FÖRUTBETALDA KOSTNADER OCH UPPLUPNA INTÄKTER

### FÖRUTBETALDA KOSTNADER OCH UPPLUPNA INTÄKTER, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Upplupna intäkter | 16 | 2 780 |
| Förutbetalda hyror | 1 183 | 568 |
| Övriga förutbetalda kostnader | 3 288 | 2 700 |
| **Summa förutbetalda kostnader och upplupna intäkter, koncernen** | **4 487** | **6 049** |

### FÖRUTBETALDA KOSTNADER OCH UPPLUPNA INTÄKTER, MODERBOLAG

| | 2022 | 2021 |
|---|---|---|
| Övriga förutbetalda kostnader | 59 | - |
| Övriga interimsfordringar | - | 50 |
| **Summa förutbetalda kostnader och upplupna intäkter, moderbolag** | **59** | **50** |

## Not 17 AKTIEKAPITAL

FlexQube har endast ett aktieslag, aktierna ger lika rätt till andel i bolagets tillgångar, resultat och eventuellt överskott vid likvidation.

| | Antal aktie | Värde per aktie |
|---|---|---|
| Antal/kvotvärde vid årets ingång | 8 233 333 | 0,1 |
| Antal/kvotvärde vid årets utgång | 8 233 333 | 0,1 |

## Not 18 LÅNGFRISTIGA SKULDER

### LÅNGFRISTIGA SKULDER, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Skulder som förfaller mellan ett och fem år från balansdagen: | 5 892 | 7 985 |
| Skulder som förfaller senare än fem år från balansdagen: | - | - |
| **Summa långfristiga skulder, koncernen** | **5 892** | **7 985** |

Posten övriga långfristiga skulder har tagits upp I balansräkningen till följd av finansiell leasing och består per aktuell balansdag enbart av finansiella leasingavtal.

## Not 19 SKULDER TILL KREDITINSTITUT

### SKULDER TILL KREDITINSTITUT, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| **Långfristiga skulder** | | |
| Skulder till kreditinstitut | 4 344 | 6 656 |
| **Kortfristiga skulder** | | |
| Skulder till kreditinstitut | 22 007 | 6 741 |
| **Summa skulder till kreditinstitut** | **26 351** | **13 397** |

## Not 20 CHECKRÄKNINGSKREDIT, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Nyttjad checkräkningskredit | 2 308 | - |
| Outnyttjad checkräkningskredit | - | 2 300 |
| **Total limit** | **2 308** | **2 300** |

## Not 21 UPPLUPNA KOSTNADER OCH FÖRUTBETALDA INTÄKTER

### UPPLUPNA KOSTNADER OCH FÖRUTBETALDA INTÄKTER, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Upplupna löner samt semesterlöner | 3 953 | 3 509 |
| Sociala avgifter | 767 | 630 |
| Förutbetalda intäkter | 8 964 | 3 701 |
| Övriga upplupna kostnader | 3 524 | 3 110 |
| **Summa upplupna kostnader och förutbetalda intäkter, koncernen** | **17 208** | **10 950** |

### UPPLUPNA KOSTNADER OCH FÖRUTBETALDA INTÄKTER, MODERBOLAG

| | 2022 | 2021 |
|---|---|---|
| Sociala avgifter | 95 | - |
| Övriga upplupna kostnader | 510 | 746 |
| **Summa upplupna kostnader och förutbetalda intäkter, moderbolag** | **605** | **746** |

## Not 22 STÄLLDA SÄKERHETER

### STÄLLDA SÄKERHETER, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Företagsinteckningar | 12 300 | 12 300 |
| Aktier i dotterföretag | 226 | 234 |

### STÄLLDA SÄKERHETER, MODERBOLAG

| | 2022 | 2021 |
|---|---|---|
| Företagsinteckningar | 12 300 | 12 300 |
| Pantsatta aktier i dotterföretag | 285 | 285 |

## Not 23 KASSAFLÖDESANALYS

### ÖVRIGA POSTER SOM INTE INGÅR I KASSAFLÖDET, KONCERNEN

| | 2022 | 2021 |
|---|---|---|
| Orealiserade valutakursdifferenser | -1 288 | 1 521 |
| Inkurans I lager | 7 284 | -82 |
| Övriga ej kassaflödespåverkande poster | -333 | 304 |
| **Summa övriga poster som inte ingår i kassaflödet, koncernen** | **5 664** | **1 744** |

### ÖVRIGA POSTER SOM INTE INGÅR I KASSAFLÖDET, MODERBOLAGET

| | 2022 | 2021 |
|---|---|---|
| Valutakursdifferens på lån i utländsk valuta | -1 699 | -2 622 |
| **Summa övriga poster som inte ingår i kassaflödet, koncernen** | **-1 699** | **-2 622** |

## Not 24 LIKVIDA MEDEL

| | 2022 | 2021 |
|---|---|---|
| Disponibla likvida medel, bankkonton hos koncernen | 30 452 | 34 925 |
| -Varav disponibla likvida medel hos moderbolag | 9 172 | 30 119 |

## Not 25 TRANSAKTIONER MED NÄRSTÅENDE

### Transaktioner med närståendebolag

Moderbolaget har tagit ett lån från bolagets styrelseordförande och VD på 4,5 MSEK vardera, räntan har fastställts till 9% på årlig basis. Samtliga transaktioner med närstående har skett till marknadsmässiga villkor.

### Inköp och försäljning mellan koncernföretag

För moderbolaget avser 0 procent (100) av årets försäljning och 0 procent (0) av årets inköp till egna dotterföretag. Försäljningen i moderbolaget avser koncernförvaltning. Övrig försäljning och inköp mellan dotterbolagen elimineras från koncernens räkenskaper i sin helhet.

## Not 26 FÖRSLAG TILL VINSTDISPOSITION

### FÖLJANDE MEDEL STÅR TILL MODERBOLAGETS FÖRFOGANDE (SEK):

| | 2022 |
|---|---|
| Balanserad vinst | 2 793 |
| Överkursfond | 144 979 |
| Årets resultat | 59 |
| **Summa följande medel står till moderbolagets förfogande** | **147 831** |

### STYRELSEN FÖRESLÅR ATT

| | |
|---|---|
| I ny räkning balanseras | 147 831 |

## Not 27 HÄNDELSER EFTER BALANSDAGEN

FlexQube har ingått ett nytt kreditfacilitetsavtal med Danske Bank A/S och ALMI om totalt 45 MSEK. 5 MSEK avser långfristigt lån och 40 MSEK tillgängliggörs via en löpande checkkredit. De nya faciliteterna skall primärt användas för att finansiera ett löpande behov av ökat rörelsekapital vid lanseringen av bolagets nya automationsprodukt FlexQube Navigator AMR under 2023.

Vid extra bolagsstämma beslutades bland annat om nytt incitamentsprogram för kommande VD i bolaget. Bolagets kommande VD tecknar 110 000 teckningsoptioner vilket tillförde bolaget c:a 1,5MSEK.

## Revisionsberättelse

Till bolagsstämman i FlexQube AB (publ), org.nr 556905-3944

### Rapport om årsredovisningen och koncernredovisningen

#### Uttalanden

Vi har utfört en revision av årsredovisningen och koncernredovisningen för FlexQube AB (publ) för år 2022. Bolagets årsredovisning och koncernredovisning ingår på sidorna 42-76 i detta dokument.

Enligt vår uppfattning har årsredovisningen och koncernredovisningen upprättats i enlighet med årsredovisningslagen och ger en i alla väsentliga avseenden rättvisande bild av moderbolagets och koncernens finansiella ställning per den 31 december 2022 och av dessas finansiella resultat och kassaflöde för året enligt årsredovisningslagen. Förvaltningsberättelsen är förenlig med årsredovisningens och koncernredovisningens övriga delar.

Vi tillstyrker därför att bolagsstämman fastställer resultaträkningen och balansräkningen för moderbolaget och koncernen.

#### Grund för uttalanden

Vi har utfört revisionen enligt International Standards on Auditing (ISA) och god revisionssed i Sverige. Vårt ansvar enligt dessa standarder beskrivs närmare i avsnittet Revisorns ansvar. Vi är oberoende i förhållande till moderbolaget och koncernen enligt god revisorssed i Sverige och har i övrigt fullgjort vårt yrkesetiska ansvar enligt dessa krav. Vi anser att de revisionsbevis vi har inhämtat är tillräckliga och ändamålsenliga som grund för våra uttalanden.

### Annan information än årsredovisningen och koncernredovisningen

Detta dokument innehåller även annan information än årsredovisningen och koncernredovisningen och återfinns på sidorna 1 - 41, 80 - 115 samt 124 - 133. Det är styrelsen och verkställande direktören som har ansvaret för denna andra information.

Vårt uttalande avseende årsredovisningen och koncernredovisningen omfattar inte denna information och vi gör inget uttalande med bestyrkande avseende denna andra information.

I samband med vår revision av årsredovisningen och koncernredovisningen är det vårt ansvar att läsa den information som identifieras ovan och överväga om informationen i väsentlig utsträckning är oförenlig med årsredovisningen och koncernredovisningen. Vid denna genomgång beaktar vi även den kunskap vi i övrigt inhämtat under revisionen samt bedömer om informationen i övrigt verkar innehålla väsentliga felaktigheter.

Om vi, baserat på det arbete som har utförts avseende denna information, drar slutsatsen att den andra informationen innehåller en väsentlig felaktighet, är vi skyldiga att rapportera detta. Vi har inget att rapportera i det avseendet.

### Styrelsens och verkställande direktörens ansvar

Det är styrelsen och verkställande direktören som har ansvaret för att årsredovisningen och koncernredovisningen upprättas och att de ger en rättvisande bild enligt årsredovisningslagen. Styrelsen och verkställande direktören ansvarar även för den interna kontroll som de bedömer är nödvändig för att upprätta en årsredovisning och koncernredovisning som inte innehåller några väsentliga felaktigheter, vare sig dessa beror på oegentligheter eller misstag.

### Rapport om andra krav enligt lagar och andra författningar

#### Uttalanden

Utöver vår revision av årsredovisningen och koncernredovisningen har vi även utfört en revision av styrelsens och verkställande direktörens förvaltning för FlexQube AB (publ) för år 2022 samt av förslaget till dispositioner beträffande bolagets vinst eller förlust.

Vi tillstyrker att bolagsstämman disponerar vinsten enligt förslaget i förvaltningsberättelsen och beviljar styrelsens ledamöter och verkställande direktören ansvarsfrihet för räkenskapsåret.

#### Grund för uttalanden

Vi har utfört revisionen enligt god revisionssed i Sverige. Vårt ansvar enligt denna beskrivs närmare i avsnittet Revisorns ansvar. Vi är oberoende i förhållande till moderbolaget och koncernen enligt god revisorssed i Sverige och har i övrigt fullgjort vårt yrkesetiska ansvar enligt dessa krav. Vi anser att de revisionsbevis vi har inhämtat är tillräckliga och ändamålsenliga som grund för våra uttalanden.

## Intygande & underskrifter

Resultat- och balansräkningar kommer att föreläggas årsstämman den 15 maj 2023 för fastställelse.

Undertecknade försäkrar att koncern- och årsredovisningen har upprättats enligt ÅRL och Bokföringsnämndens allmänna råd BFNAR 2012:1 Årsredovisning och koncernredovisning (K3), respektive god redovisningssed och ger en rättvisande översikt av koncernens och moderbolagets verksamhet, ställning och resultat.

Förvaltningsberättelsen för koncernen och moderbolaget ger en rättvisande översikt över utvecklingen av koncernens och företagets verksamhet, ställning och resultat samt beskriver väsentliga risker och osäkerhetsfaktorer som moderbolaget och de företag som ingår i koncernen står inför.

Göteborg den dag som framgår av vår elektroniska signatur.

**Christian Thiel**  
STYRELSEORDFÖRANDE

**Anders Ströby**  
STYRELSELEDAMOT

**Mikael Bluhme**  
STYRELSELEDAMOT

**Per Augustsson**  
STYRELSELEDAMOT

**Anders Fogelberg**  
VERKSTÄLLANDE DIREKTÖR

Vår revisionsberättelse har lämnats den dag som framgår av vår elektronisk signatur.

**PricewaterhouseCoopers AB**

**Johan Palmgren**  
AUKTORISERAD REVISOR

# Hållbarhet

## Globala målen för hållbar utveckling

I denna rapport kommer vi att beskriva våra insatser för att minska vår miljöpåverkan, främja social rättvisa och öka vår hållbarhet på lång sikt. Vi hoppas att denna redovisning kommer att ge en tydlig bild av våra framsteg och vår ambition att bidra till en mer hållbar framtid.

På FlexQube har vi under det senaste året fortsatt att komma närmare vår strävan efter en hållbar framtid. Efter att ha identifierat sex relevanta mål för hållbar utveckling förra året tillsammans med Almi, har vi nu fortsatt undersöka möjliga åtgärder för att integrera dessa mål i vår verksamhet.

### Våra sex FN-mål:

- **Mål 3**: Säkerställa hälsosamma liv och främja välbefinnande för alla i alla åldrar
- **Mål 5**: Uppnå jämställdhet och alla kvinnors och flickors egenmakt
- **Mål 8**: Verka för varaktig, inkluderande och hållbar ekonomisk tillväxt, full och produktiv sysselsättning med anständiga arbetsvillkor för alla
- **Mål 9**: Bygga motståndskraftig infrastruktur, verka för en inkluderande och hållbar industrialisering samt främja innovation
- **Mål 12**: Säkerställa hållbara konsumtions- och produktionsmönster
- **Mål 13**: Vidta omedelbara åtgärder för att bekämpa klimatförändringarna och dess konsekvenser

## Vår Hållbarhetsstrategi

Hållbarhet är fortfarande en viktig fråga för FlexQube. Vi har fortsatt att ta oss an kriterier för miljö, socialt ansvar och styrning som en del av vår hållbarhetsstrategi. Vårt hållbarhetsarbete är en ständigt pågående process och vi strävar efter att göra förbättringar varje dag.

### Våra mål

#### Miljömässig hållbarhet
- Fortsätta utvecklingen av ergonomisk, innovativ och hållbar materialhantering för tillverkare och lager samt minska livscykelpåverkan
- Utveckla och implementera eftermarknadsservice för att upprätthålla en ansvarsfull leverantörskedja och utvärdera viktiga leverantörer mot vår bedömning
- Fortsätta minska våra koldioxidutsläpp
- Fortsätta att kartlägga vår energiförbrukning i hela verksamheten
- Fortsätta att minska vårt avfall från våra tillverkningsplatser

#### Socialt ansvar
- Hjälpa våra kunder att uppnå bättre säkerhet på verkstadsgolvet
- Arbeta aktivt med personlig utveckling och kunskapsöverföring genom utbildning, workshops och seminarier
- Sträva mot jämställdhet inom organisationen
- Uppnå en diversifierad arbetskraft som representerar de samhällen där vi bor och arbetar
- Arbeta aktivt med organisationsutveckling genom att mäta eNPS

#### Styrning
- Främja hållbarhet genom utbildning, partnerskap och vår uppförandekod

## Miljömässig Hållbarhet

Klimatförändringarna är verkliga och det är vår uppgift att bidra till en hållbar framtid genom materialhantering och innovation. Vi fortsätter att vara fast beslutna om att minska vår påverkan på miljön.

### Industri, Innovation & Infrastruktur

Att bygga en innovativ industrisektor är avgörande för att möjliggöra att produkter erbjuds till konsumenterna med hållbarhet i åtanke. Många av FlexQubes kunder är stora internationella företag med produkter i världsklass samt tillverkning och verksamhet globalt.

**60 000+** levererade vagnsystem

**39+** aktiva och pågående patent över hela världen

### Ansvarsfull konsumtion och produktion

#### Konsumtion
Vårt åtagande för att minska klimatutsläppen genom upcycling och återanvändning av befintliga materialhanteringsvagnar har under 2022 fortsatt och utvecklats. Med FlexQubes patenterade modulära koncept, och eftermarknadsservice, hjälper vi kunder att reparera och modifiera sina befintliga vagnar istället för att köpa nya när processer och material förändras.

#### Leverantörer
Vi har identifierat nya affärsmål som vi kontinuerligt kommer att implementera i hela vår organisation framöver. Under året 2022 har vi åtagit oss att stärka våra relationer med leverantörer som delar liknande värderingar som oss när det gäller säkerhet och miljöansvar.

**Syftet med vår leverantörsbedömning är att betygsätta:**
- Kvalitet och leveransprestanda
- Miljöaspekter
- Hälsa och säkerhet
- Mänskliga rättigheter
- Utvärderingsmetod
- Affärsetik

**Bedömningsskala:**
- 0%: Inte kvalificerad
- 70%: Obligatorisk handlingsplan
- 85-100%: Kvalificerad

### Klimatåtgärder

#### Material
Vi är engagerade i ansvarsfulla inköp av alla råvaror som används för att tillverka våra grundläggande byggstenar. Vårt kärnmaterial är högkvalitativt stål som säkerställer robusta och hållbara lösningar.

#### Utsläpp/Transport
Under det senaste året har vi fortsatt att utveckla vårt FlexQube modulkoncept, som gör det möjligt att skicka vagnar som "platta paket" för att monteras på plats. Detta minskar behovet av transporter med 70 procent.

#### Elförbrukning
Under 2022 bytte vi lokaler i Sverige:
- Tidigare lokal (januari-september): 53 503 kWh totalt, i genomsnitt 5 945 kWh per månad
- Ny lokal (augusti-december): 33 772 kWh totalt, i genomsnitt 6 754 kWh per månad

#### Vatten
I vår anläggning i USA användes under 2022 i snitt 4 764 gallons vatten per månad, med en total förbrukning på 57 163 gallons (216 385 liter).

#### Avfall
Under perioden 1 januari till 31 december 2022 hade vi en total återvinning på 27 350 kg som inte var farligt avfall.

# Miljöhållbarhet

## Anpassad design

Vi ser många hållbara förbättringar i hela vår organisation, varav en är att vi kommer att fortsätta att förbättra våra produkters miljöpåverkan genom att designa bättre och mer miljövänliga produkter. Under årens lopp har vi designat mer än 6 000

## Miljödata

- **Total förbrukning**: 190 MWh
- **Transporter**: Väg & hav 99% under 2022, Flyg <1%
- **Avfall**: 27 ton, varav 1,2% var farligt avfall 2022 (2% under 2021)
- **Återvinning**: Avfall skickat för återvinning från vår montering i Sverige

---

# Socialt ansvar

Våra medarbetare är nyckeln till vår framgång, och vi tror att mångfald, jämlikhet och inkludering driver verkliga framsteg. Vi är ett team av innovativa, engagerade proffs som förenas av vårt uppdrag; att göra våra kunder världsledande inom intralogistik.

## Våra mål

### Miljöhållbarhet
- Fortsätta utvecklingen av ergonomisk, innovativ och hållbar materialhantering för tillverkare och lager samt minska livscykelpåverkan
- Utveckla och implementera eftermarknadsservice för att upprätthålla en ansvarsfull leverantörskedja och utvärdera viktiga leverantörer mot vår bedömning
- Fortsätta minska våra koldioxidutsläpp
- Fortsätta att kartlägga vår energiförbrukning i hela verksamheten
- Fortsätta att minska vårt avfall från våra tillverkningsplatser

### Socialt ansvar
- Hjälpa våra kunder att uppnå bättre säkerhet på verkstadsgolvet
- Arbeta aktivt med personlig utveckling och kunskapsöverföring genom utbildning, workshops och seminarier
- Sträva mot jämställdhet inom organisationen
- Uppnå en diversifierad arbetskraft som representerar de samhällen där vi bor och arbetar
- Arbeta aktivt med organisationsutveckling genom att mäta eNPS

### Styrning
- Främja hållbarhet genom utbildning, partnerskap och vår uppförandekod

---

# Social hållbarhet

## Hälsa och säkerhet

God hälsa och välbefinnande är avgörande för att förbättra våra anställdas välbefinnande. Vi är bestämda att ta itu med eventuella drogmissbruk som uppstår och arbetar ständigt för att säkerställa en säker och hälsosam miljö.

Vi följer ISO 45001-standarden för att säkerställa att våra säkerhetsrutiner är uppdaterade. Vi följer upp antalet rapporterade skador som uppstår på plats och har mindre än 10 rapporterade skador, varav ingen krävde frånvaro från arbetet.

**Nyckeltal:**
- **<10** mindre olyckor
- **29** Learnifier-kurser
- **3,5 h** utbildning i Learnifier per anställd

## Träning och utbildning

FlexQube har ett ombordstigningsprogram för nyanställda som täcker alla aspekter av företaget, inklusive erbjudanden, värderingar, kunder och hållbarhetsarbete. Varje chef har årliga P&D-diskussioner med sina anställda för att sätta upp mål, bedöma kompetenser och identifiera utbildningsbehov.

Vi använder Learnifier för att spåra anställdas deltagande i kompetensutveckling, och i genomsnitt har 92% av alla anställda deltagit i 3,5 utbildningstimmar per anställd.

## Jämställdhet

Jämställdhet mellan könen är inte bara en grundläggande mänsklig rättighet, utan en nödvändig grundsten för en fredlig, välmående och hållbar värld. Vårt mål för att uppnå detta är genom lika möjligheter för kvinnor, liksom genom en mångfaldig och inkluderande kultur.

### Personalstatistik 2022 vs 2021

**Alla anställda:**
- 2022: Kvinnor 26%, Män 74% (Medelålder 35)
- 2021: Kvinnor 25%, Män 75% (Medelålder 36)

**Ledning:**
- 2022: Kvinnor 29%, Män 71%
- 2021: Kvinnor 17%, Män 83%

**Styrelse:**
- Både 2022 och 2021: Kvinnor 0%, Män 100%

### Arbetskraft per region

**2022:**
- USA 47%
- Sverige 40%
- Mexiko 9%
- UK 2%
- Tyskland 2%

**2021:**
- Sverige 46%
- USA 42%
- Mexiko 8%
- UK 2%
- Tyskland 2%

## Medarbetarengagemang

Vi värderar kontinuerligt våra anställdas välbefinnande och förtroende genom vår Employee Net Promoter Score (eNPS) undersökning. Vi anser att våra anställda är grunden till FlexQubes framgång och vår största prioritet är att se till att de trivs och mår bra.

## Fördomsfrihet

På FlexQube tar vi all form av mobbning på stort allvar. För att säkerställa detta har vi etablerat rutiner för att rapportera incidenter som ska följas upp av en ansvarig chef med stöd av HR. Vi har inte några kända fall som fortfarande är föremål för åtgärder och alla ärenden hanteras med absolut konfidentialitet.

## Utveckling av våra kärnvärden

På FlexQube är vår största tillgång våra medarbetare. Våra kärnvärden är av yttersta vikt och var avgörande för att sätta vår nya strategiska inriktning för 2023-2026.

**BRANSCHLEDANDE FÖRTROENDE MOD KUNDFOKUSERADE**

---

# Styrning och ekonomisk hållbarhet

## Anständigt arbete och ekonomisk tillväxt

Sedan 2021 har FlexQube integrerat hållbarhet i affärsmodellen och den strategiska inriktningen 2022-2026. Vår strategi har sex riktningar: Stärka arbetet med befintliga kunder, accelerera robotförsäljningen, öka effektivitet i operations, lönsamhet och kassaflödesoptimering, utveckla personal och företagskultur, samt service och eftermarknad.

**Vår mission:** Vi existerar för att göra våra kunder världsledande i internlogistik.

## Våra intressenter

### Anställda
Vi uppmuntrar feedback från våra anställda genom regelbundna företagsmöten, vår eNPS och våra prestations- och utvecklingsrecensioner.

### Kunder
Vi söker kundfeedback genom kontinuerlig dialog, produktutbildning, distributörsmöten och uppföljningar samt olika kundundersökningar.

### Leverantörer
Vi har ett nära samarbete med våra leverantörer för att förbättra våra resultat och skapa en mer hållbar värdekedja.

### Investerare
Vi tror på en öppen och enkel kommunikation med våra investerare. Vi är öppna med våra finansiella resultat, styrning och strategier.

### Myndigheter
Våra initiativ för myndighetsrelationer syftar till att utbilda och informera tjänstemän om en rad politiska frågor som är viktiga för vår verksamhet.

### Samhällen
Vi försöker påverka de lokala samhällen där våra anläggningar ligger positivt.

## Mänskliga rättigheter

FlexQube är engagerad i gott medborgarskap och främjande av bättre arbetsvillkor för alla. År 2022 gav vårt team i Mexiko tillbaka till sitt samhälle med en donation av leksaker till ett lokalt barnhem. För att ge stöd till de drabbade invånarna under kriget i Ukraina donerade FlexQube även en summa till USSSE.

## Lika lön

Lika lön för likvärdigt arbete. I vår årliga löneöversyn bedömer vi om det finns skillnader mellan lönenivåer. Vi har 3 nivåer av karriärsteg: bidra självständigt, bidra genom andra och bidra strategiskt.

## Respektera datasekretess

Vi respekterar integriteten för personer från vilka vi samlar in personuppgifter och vi har vidtagit åtgärder under 2022 för att skydda våra uppgifter på ett än säkrare sätt.

## Antikorruption

Vår uppförandekod definierar ramarna för vår affärsetik och våra förväntningar på medarbetare, partners och leverantörer. Vi har nolltolerans mot alla typer av korruption och har interna kontrollåtgärder för att förebygga och identifiera om det skulle ske.

---

# Bolagsstyrning

## Ordförande har ordet

### Resan fortsätter – FlexQube visar urstark försäljningsutveckling

# Etablerad position som robotföretag

Tydliga mål, fortsatt fokus, finslipade säljstrategier, ökade investeringar samt tid spenderad på vår robotteknik resulterar i att våra ambitioner om tillväxt efterföljs även 2022. Vår försäljning växte med ca 82 procent och vi adderade ca 92 MSEK till vår försäljning jämfört med året innan. Vår riktning är tydlig och för varje dag som passerar befäster vi vår position som ett innovativt och ledande internlogistikföretag med högst relevant produktportfölj. Jag vågar påstå att vi är det enda bolaget som har en produktportfölj som fullt ut kan erbjuda lösningar för materialpresentation och -transport oberoende kundens mognadsgrad.

Det har minst sagt varit ett innehållsrikt år, inte minst utvecklingen sett till försäljnings-utvecklingen men också att det var första gången i bolagets historia som arbetet med ett VD-skifte inleddes, vilket framgångsrikt ledde till rekryteringen av Mårten Frostne som tar över stafett-pinnen från Anders Fogelberg som med storhet manövrerat FlexQube sedan starten.

## Kraftigt ökande försäljningsvolymer och ännu större kundfokus

Under 2022 utvecklades FlexQubes försäljnings-volymer ännu starkare än under 2021 tack vare vårt unika koncept, kundfokus och hårda arbete. Under årets första nio månader var vi dessutom lönsamma, trots aggressiv tillväxt och trots de kostnader som varit nödvändiga för att accelerera bolagets robot-utveckling. Vi är glada att se lönsamhets-

men det finns mycket kvar att arbeta med för att förstärka rörelsemarginalen ytterligare i takt med att vi växer. Även om fokuset på att utveckla våra marginaler är stort så är lönsamhet på nedersta raden inte primärt fokus de kommande 2 åren, alternativ-kostnaden bedöms fortsatt för stor för att inte investera så mycket som möjligt i oss själva och vår utveckling i marknaden.

Fortsättningsvis så premierar vi att bygga långsiktiga och partnerlika relationer med våra kunder, särskilt med de kunder av strategisk karaktär där fördelarna med att hålla oss nära är både stora och många. Att arbeta partnermässigt kommer bli än mer aktuellt i takt med bolagets ökande robotisering och vi ser nu exempelvis mycket fram emot att arbeta med Tesla i deras nya fabrik i Mexiko, där vi har ett säljkontor endast 45 minuter från deras fabrik. Vi spår att våra

robotprodukter kommer utgöra en ännu större del av vårt projektinflöde i samband med att vi får ut allt fler robotar till kunder och dess unika förmåga bevisas i fler skarpa lägen. Marknaden för automatisering och robotteknik ökar i en snabb takt och vårt helhets-koncept fortsätter att sprida sig över branscher. Detta accelererar vår intelligens och förståelse för olika typer av kunders behov samt driver oss till att bli mer diversifierade i vårt erbjudande och utveckling.

Det stegrande projektinflödet för eQart® Line ger ett än mer stärkt självförtroende som robotföretag. Fram-tidstron för FlexQube är orubblig, men vi är inget undantag från att känna effekterna av de osäkerheter som råder globalt. Under de sista månaderna av 2022 kunde en generell fördröjning i försäljningsproces-sen från offert till order identifieras, vilket med stor sannolikhet grundar sig i externa osäkerheter. Det är en tid som kräver balans och att vi lyhört har örat mot rälsen för att kunna ta väl avvägda beslut i hur vi gasar och bromsar verksamheten framåt. Det momentum som byggts upp i bolagets robotförsäljning vill vi i allra högsta grad tillvarata, men det kan inte ske på för stor bekostnad av annat som är av prioritet för bolaget som helhet.

Under de första månaderna av 2023 har vi dock märkt ett kraftigt ökande projektinflöde. Även om det finns osäkerheter i marknaden så har våra kunder otroliga investeringsbehov för att förbättra sin logistik. Vi kommer fortfarande från jämförelsevis låga tal och med rätt samt framåtlutat fokus ska vi trots yttre osäkerheter kunna växa vår verksamhet.

## Stora kliv för bolagets robot-utveckling

Robotutveckling är numera en del i FlexQubes DNA. Vi har passerat brytpunkten från att vara ett mekaniskt internlogistikbolag till att vara ett robotteknikbolag för internlogistik, oavsett kundens mognadsgrad. Vår riktning dit vi styr bolaget är tydlig, utan att göra avkall på vårt flexibla och anpassningsbara helhetskoncept. Nuvarande och kommande års fokus på FlexQube Navigator AMR är nästa stora steg i vår robotresa, efter eQart® Line. FlexQube Navigator AMR fyller en tydlig lucka ute i marknaden och det är en produkt som tillåter oss att jobba integrerat och oerhört kund-

orienterad samtidigt som vi får maximal användning av FlexQube-konceptet. Vi kan differentiera oss på ett sätt som inte är möjligt i samma utsträckning med traditionella produkter och vi är av övertygelsen att vi lyckats skapa en automationsprodukt för intern-logistik i tillverkningsindustrin. Något som ingen lyckats med tidigare. I början av 2023 installerar vi våra första FlexQube Navigator AMR i ett pilotprojekt med en av de största biltillverkarna och vi ser med spänning fram emot vad det kommer leda till. Ytterligare pilotprojekt planeras från andra kvartalet och framåt men där de reella försäljningsvolymerna inte beräknas förrän tidi-gast under fjärde kvartalet 2023.

Det är lätt att underskatta tiden det tar att bygga reella volymer med högteknologiska robotar, alltifrån ifrån att tekniken ska fungera till 100 procent till att få rätt resurser på plats internt för exempelvis volym-produktion eller service och eftermarknad. Därför är vårt fokus att få alla delar på plats för att därefter verkligen kunna skala upp volymerna.

För varje dag som går förstärks vår tro på vår första robotprodukt, eQart® Line. Erbjudandet och problemen den löser på ett unikt, enkelt och kostnads-effektivt sätt är bättre än vad vi hade hoppats på och eQart® Line, tillsammans med FlexQube Navigator AMR, kommer utgöra ett stort fokus för oss kommande åren.

## En organisatoriskt händelserik tid

En ökad efterfrågan på våra robotar och vagnar leder till ett internt ökat behov av resurser, vilket innebär att vår organisation växer. En växande organisation, särskilt en snabbt växande sådan, ställer högre krav på strategisk styrning likväl som standardiserade rutiner och processer för att kunna agera optimalt och skapa så mycket värde som möjligt hela tiden. Redan under föregående år var det tydligt att vi hade en ny typ av organisatoriskt arbete framför oss med att rusta FlexQube för att kunna agera som ett större bolag, däribland var kulturbyggande och än mer intrimmade processer några prioriteter. Kulturbyggande är dock ett arbete som inte är klart på ett år utan det är dynamiskt och fortsätter konstant i takt med tillväxten, men en av nyckelaktiviteterna under det gånga året har varit att stärka HR-avdelningen som tillsammans med

ledningen ska driva det interna arbetet väl planlagt och resultatriktat framåt. Det är en grupp drivna människor med höga ambitioner som arbetar hos oss och det är glädjande att de vill investera i sin karriär på FlexQube. Allt fler anställda tecknar optioner i bolag-et vilket är ett kvitto på att de verkligen tror på det vi gör och på resan vi har framför oss. Det här är oerhört inspirerande för mig och resten av styrelsen. I strategin framåt för verksamheten ingår också att anpassa organisationen för den ökade robot-försäljningen vi uppnår. Idag är vi jämförelsevis teknik-tunga, vilket har varit viktigt för att nå dit vi är idag med våra produkter och erbjudanden, och som en effekt av det framgångsrika arbetet kommer behovet av försäljning, produktion och eftermarknad vara ännu större framgent, vilket bör speglas i hur vi växer och formerar oss.

En organisatoriskt händelserik tid, som avsnittets rubrik lyder, syftar naturligtvis också till rekryteringen av en ny VD för FlexQube. Det är första gången i bolagets historia som stafettpinnen för positionen överlåts, och att ta steget från en VD som varit med och grundat bolaget till en extern tillgång innebär så klart förändringar för organisationen som helhet men det är en stor möjlighet att vidareutveckla oss. Kommande struktur ger en möjlighet att företa sig FlexQubes utmaningar och möjligheter med nya ögon, ofärgade av bolagets historik och med andra erfarenheter samt kvalifikationer. Detta i kombination med att ha kvar Anders kompetens och kunskap i bolaget är en mycket gynnsam förutsättning för ett VD-skifte. Tillträdet sker under juni 2023 och överlämningen startade redan under februari för att optimera förutsättningarna för skiftet. Jag tillsammans med övriga styrelsen ser mycket fram emot att arbeta minst lika resultatinriktat i den nya strukturen.

## Fortsatt stark och engagerad styrelse i nära samarbete med VD och ledning

Vinsterna är stora i att kontinuerligt arbeta för att stärka samspelet mellan styrelse, VD:n och dennes ledning. Det är min fasta övertygelse. Som styrelse-ordförande anser jag det vara min viktigaste uppgift att få fram och maximera den enade kraften som kommer

ur samarbetet. Den tydliga distinktionen att styrelsen driver diskussioner och beslut av strategisk karaktär för att sedan stötta ledningen i operativt genomförande, är grundläggande för att kunna agera proaktivt och målinriktat i hur bolaget styrs. Den underliggande gemensamma strävan är att varje dag skapa största möjliga värdeutveckling för FlexQube och dess aktieägare, vilket ställer krav på att varje individ stöttas men också hålls ansvarig gentemot förväntningar och mål. Det ställer också krav på att det vid varje givet tillfälle finns en plan och strategi för att nå våra mål och bemöta situationer vi kan förutse, men framför allt för att hantera de situationer vi inte kan förutse. Det gemensamma arbetet präglas av ett tydligt framtids-fokus och tyngdpunkten i de frekventa möten som hålls är diskussioner och beslutsfattande om hur verksamheten och affären kan tas framåt ur ett strategiskt perspektiv.

Likväl som målmedvetet samarbete mellan styrelse, VD:n och dennes ledning är fundamentalt, är det också en given prioritet inom respektive grupp. Under det gånga året gav rekryteringsprocessen av en ny VD starka bevis på hur styrelsen är en sammansättning av personer med kompletterande erfarenheter, kompetenser och personligheter, där arbetet tillsammans fungerat framgångsrikt. Att rekrytera en ny VD är bland det viktigaste man kan göra i sitt arbete som styrelsemedlem och rekryteringsprocessen visade på ett dynamiskt samspel som grupp. Samtliga har bidragit med relevant expertis för att kvalitetssäkra en process som leder till bästa möjliga resultat. Även om vår nya VD ännu inte är på plats, råder det konsensus om att han är rätt person för jobbet. Styrelsen och Mårten har en mycket stark tro på varandra och vad vi skall kunna åstadkomma tillsammans i framtiden.

Som med alla VD-byten innebär det att ett nytt sam-spel ska ta fäste, både inom ledningsgruppen och tillsammans med styrelsen. Vi ser med ljus framtidstro på detta och är trygga i att nya dynamiska relationer kommer att skapas. Med det sagt vill jag å styrelsens vägnar tacka Anders Fogelberg för allt han uträttat i sin roll som VD för FlexQube och för det fina samarbete som bedrivits. Även om det här fortfarande bara är början på vår resa har FlexQube under Anders Fogelberg utvecklas otroligt väl på endast några år,

och han har onekligen varit en central del i det. Det finns många fantastiska historier med Anders genom åren som idag är en del av bolagets DNA. En specifik händelse som jag kommer bära med mig är när Anders med familjen utan någon som helst tvekan flyttade till USA 2014 när vi insåg att Nord-amerika är rätt marknad för oss för snabb avkastning på våra begränsade medel då. Utan någon social plattform flyttade han med sin fru och deras nyfödda son för att börja bygga våra försäljningsvolymer från 0 till att nu representera ca 150 MSEK och där vårt varumärke växt sig väldigt starkt. Lika mycket som jag vill tacka Anders för hans otroliga arbete vill jag också tacka Anders fru Carolina Fogelberg för det fantastiska arbete hon har lagt ner för att få allt att fungera under alla år. Stort tack Anders och Carolina!

I sammanhanget ska kommas ihåg att bara för att vi byter VD så försvinner inte Anders utan han kommer fortsatt arbeta kvar i företaget och bidra till att utveckla bolaget till nya höjder med sina ovärderliga kunskaper och kvaliteter. Laguppställningen är därmed ännu starkare nu när vi påbörjar nästa fas i bolaget.

## Framtiden är vår

Även om vi hade en fantastisk utveckling under 2022 så har vi som jag nämnt tidigare, fortsatt bara skrapat på ytan sett till marknadens storlek och potential. Det finns oerhörda investeringsbehov där ute för en optimerad logistik som möter ständigt föränderliga variabler. Vår verksamhet frodas i en miljö som ställer krav på innovativa och flexibla lösningar, vilket innebär att vi har både förutsättningarna på marknaden samt verktygen internt att skala upp FlexQube till något riktigt stort. Framtidstron är stark och även om ingen väg är utan hinder så är det generella synsättet att de hinder vi stöter på bör betraktas som utvecklande möjligheter. En grundbult i FlexQubes existens är att varje dag arbeta för att som bolag vara lite bättre än vi var igår. Det kräver hårt arbete, vilja och tillförsikt vilket är något jag kan skriva under på att jag ser varje dag i den här verksamheten. Jag har sagt det förut och jag säger det igen – vi står redo med uppkavlade ärmar för att ta oss an resan framåt, redo att skapa maximal värdeutveckling för våra aktieägare.

Göteborg, 24 april, 2023

**Christian Thiel**  
STYRELSEORDFÖRANDE, FLEXQUBE AB (PUBL)

---

## Styrelsens årsplanering

Styrelsearbetet följer en struktur med fasta och återkommande ärenden i huvudsak enligt följande plan.

### FEBRUARI
**Styrelsen**
- Bokslutskommuniké
- Årsredovisning
- Förslag till utdelning
- Förberedelse inför årsstämman

### MARS
**Styrelsen**
- Bolagsstyrningsrapport
- Styrelseutvärdering
- Rapportering från revisor
- Utvärdering revisorer
- Revisionsplan

### APRIL/MAJ
**Styrelsen**
- Kvartalsrapport första kvartalet
- Revisionsplan
- Utvärdering av incentivprogram och program att framläggas på årstämma

**Konstituerande styrelsemöte**
- Styrelsens och utskottens arbetsordningar, VD-instruktioner
- Utvärdering av behovet av respektive revisionsutskott och ersättningsutskott

**Ersättningsutskott**
- Övergripande löne- och villkorspolicy för koncernen
- Ersättning till ledande befattningshavare

# Bolagsstyrningsrapport

## Översikt

FlexQube AB (publ) är ett publikt svenskt bolag vars aktier är noterade på Nasdaq First North Stockholm. Bolaget är en global leverantör av flexibla och robusta industrivagnar och robotar inom materialhantering. Standardiserade gränssnitt och modulära byggblock möjliggör en unik, effektiv och skalbar designprocess där kunderna får tillgång till unika lösningar.

FlexQubes bolagsstyrning beskriver hur ägarna, genom bolagsstämman, och styrelsen styr bolaget samt säkerställer att VD och bolagsledningen skapar värde och hanterar riskerna i verksamheten.

## Bolagsordning

Bolagets firma är enligt bolagsordningen FlexQube AB (publ) och bolaget är publikt. Styrelsen har sitt säte i Västra Götalands län, Mölndals kommun. Bolaget ska, direkt eller indirekt, utveckla, tillverka och marknadsföra materialställ samt lås- och kopplingsanordningar för tillverknings- och byggindustrin samt detaljhandeln. Bolaget ska även utföra konsulttjänster inom produktions- och produktutveckling med fokus på maskinteknik, samt idka därmed förenlig verksamhet.

## Aktieägare

FlexQube-koncernen består av fem bolag. Moderbolag i koncernen är det svenska publika aktiebolaget FlexQube AB (publ), vars aktier är noterade på Nasdaq First North Stockholm.

Aktiekapitalet i FlexQube representeras av stamaktier. Varje aktie berättigar till en röst. Samtliga aktier medför samma rätt till andel av bolagets tillgångar och vinst. Antal aktier uppgår till 8 233 333. Per den 31 december 2022 uppgick antalet aktieägare till ca 1 900 stycken.

De större huvudaktieägarna i FlexQube utövar en aktiv ägarroll. Styrelse och koncernledning kontrollerar sammanlagt 71,5 procent av kapital och röster i bolaget. Bolagsordningen innehåller inget förbehåll om rösträttsbegränsning.

## Bolagsstämma

Aktieägares rätt att besluta i FlexQubes angelägenheter utövas genom det högsta beslutande organet årsstämma eller extra bolagsstämma. Stämman beslutar till exempel om ändringar i bolagsordning, styrelse- och revisorsval, fastställelse av resultat- och balansräkning, ansvarsfrihet för styrelse och verkställande direktör samt disposition av vinst eller förlust.

Enligt FlexQube AB:s bolagsordning ska kallelse till bolagsstämma ske genom annonsering i Post- och Inrikes Tidningar och på Bolagets webbplats. Att kallelse skett ska samtidigt annonseras i Dagens Industri.

### Rätt att delta i bolagsstämma

För att få delta i bolagsstämma ska aktieägare anmäla sig till FlexQube senast den dag som anges i kallelsen till stämman. Denna dag får ej vara söndag, annan allmän helgdag, lördag, midsommarafton, julafton eller nyårsafton och ej infalla tidigare än femte vardagen före stämman.

### Initiativ från aktieägare

Aktieägare som önskar få ett ärende behandlat på bolagsstämman måste framställa en skriftlig begäran till bolagets styrelse. En sådan begäran ska i normala fall vara styrelsen tillhanda senast fyra veckor före bolagsstämman.

### Årsstämma 2022

Årsstämma 2022 hölls den 12 maj genom poströstning. Sammanlagt var 72,8 procent av totalt antal aktier respektive röster närvarande. Räkenskaperna för 2022 fastställdes och styrelseledamöterna och VD beviljades ansvarsfrihet. Beslut togs också om val av styrelseledamöter, styrelseordförande och revisorer samt arvode till styrelse och revisorer. Det beslutades även om uppdatering av bolagsordning samt bemyndigande för styrelsen att besluta om nyemission upp till 10 procent.

### Årsstämma 2023

Årsstämma hålls 15 maj 2023 på bolagets huvudkontor i Mölndal.

## Valberedning

Bolagsstämman i FlexQube har för närvarande ingen valberedning.

## Styrelse

Styrelsen är efter bolagsstämman FlexQubes högsta beslutande organ. Det är styrelsen som ska svara för bolagets organisation och förvaltningen av bolagets angelägenheter till exempel genom att fastställa mål och strategi, säkerställa rutiner och system för uppföljning av de fastslagna målen, fortlöpande bedöma FlexQubes ekonomiska situation samt utvärdera den operativa ledningen. Det är vidare styrelsens ansvar att säkerställa att korrekt information ges till bolagets intressenter, att bolaget följer lagar och regler samt att Bolaget tar fram och implementerar interna policyer och etiska riktlinjer. Styrelsen utser även FlexQubes verkställande direktör och fastställer lön och annan ersättning till denne utifrån gällande riktlinjer.

### Styrelsens sammansättning

Styrelsens bolagsstämmovalda ledamöter väljs årligen av årsstämman för tiden intill dess nästa årsstämma hållits. Styrelsen ska enligt bolagsordningen bestå av lägst tre och högst sex bolagsstämmovalda ordinarie ledamöter utan suppleanter.

Styrelsen har under 2022 bestått av fyra ledamöter som är valda på ett år av årsstämman. För närvarande består bolagets styrelse av de ordinarie stämmovalda ledamöterna Christian Thiel, styrelseordförande, Per Augustsson, Mikael Bluhme och Anders Ströby. VD Anders Fogelberg ingår inte i styrelsen, men deltar som föredragande vid alla möten.

| Namn | Antal Aktier (T) | Närvaro styrelsemöten | Oberoende till ägare respektive till bolaget |
|------|------------------|----------------------|---------------------------------------------|
| Christian Thiel | 1 945 | 18/18 | Nej |
| Per Augustsson | 1 458 | 18/18 | Nej |
| Anders Ströby | 70 | 18/18 | Ja |
| Mikael Bluhme | 31 | 18/18 | Ja |

### Styrelsens ordförande

Styrelsens ordförande har till uppgift att tillse att styrelsens arbete bedrivs effektivt och att styrelsen fullgör sina åligganden. Ordföranden ska särskilt organisera och leda styrelsens arbete för att skapa bästa möjliga förutsättningar för styrelsens arbete.

Det är ordförandens uppgift att tillse att ny styrelseledamot genomgår erforderlig introduktionsutbildning samt den utbildning i övrigt som styrelseordföranden och ledamoten gemensamt finner lämplig, tillse att styrelsen fortlöpande uppdaterar och fördjupar sina kunskaper om bolaget, tillse att styrelsen erhåller tillfredsställande information och beslutsunderlag för sitt arbete, fastställa förslag till dagordning för styrelsens sammanträden efter samråd med verkställande direktören, kontrollera att styrelsens beslut verkställs samt tillse att styrelsens arbete årligen utvärderas.

Styrelseordföranden ansvarar för kontakter med ägarna i ägarfrågor och för att förmedla synpunkter från ägarna till styrelsen. Ordföranden deltar inte i det operativa arbetet inom bolaget. Han ingår inte heller i koncernledningen.

### Styrelsearbetet

Styrelsen följer en skriftlig arbetsordning som ses över årligen och fastställs på det konstituerande styrelsemötet. Arbetsordningen reglerar bland annat styrelsens arbetsformer, arbetsuppgifter, beslutsordning inom Bolaget, styrelsens mötesordning, ordförandens arbetsuppgifter samt en lämplig arbetsfördelning mellan styrelsen och verkställande direktören. Instruktion avseende ekonomisk rapportering och instruktion till verkställande direktör fastställs också i samband med det konstituerande styrelsemötet.

Styrelsens arbete bedrivs också utifrån en årlig sammanträdesplan som tillgodoser styrelsens behov av information. Sammanträdesplanen ska utformas så att sammanträden sammanfaller med framställningen av kvartalsrapporter, bokslut samt bokslutskommuniké.

De på styrelsen ankommande kontrollfrågorna handhas av styrelsen i dess helhet. Styrelseordföranden och verkställande direktören har vid sidan av styrelsemötena en löpande dialog kring förvaltningen av FlexQube. Styrelsen sammanträder efter en i förväg beslutad sammanträdesplan och ska mellan varje årsstämma hålla minst fem ordinarie styrelsemöten. Utöver dessa möten kan extra möten arrangeras för behandling av frågor som inte kan hänskjutas till något av de ordinarie mötena. Under räkenskapsåret 2023 har styrelsen hållit 18 protokollförda möten, varav ett konstituerande möte. Därutöver har styrelse och ledning genomfört ett antal arbetsmöten för uppdatering med hänsyn till den utmanande marknadssituationen.

## Utskott

Styrelsen har tillsatt ett revisionsutskott och ett ersättningsutskott.

### Revisionsutskott

Styrelsen utgör revisionsutskott med Christian Thiel som ordförande. Revisionsutskottets huvudsakliga uppgift är att övervaka FlexQubes finansiella rapportering, övervaka effektiviteten i bolagets interna kontroll, internrevision och riskhantering med avseende på den finansiella rapporteringen samt lämna rekommendationer och förslag för att säkerställa rapporteringens tillförlitlighet. Revisionsutskottet ska, i samverkan med FlexQubes revisor, säkerställa revisionsplanering, uppföljning samt revisionsstatus inför årsredovisning och koncernredovisning samt om slutsatserna i Revisorsnämndens kvalitetskontroll.

Därutöver ska revisionsutskottet övervaka revisorns opartiskhet och självständighet och särskilt, i förekommande fall, förhandsgodkänna när revisorn tillhandahåller bolaget andra tjänster än revisionstjänster.

### Ersättningsutskott

Ersättningsutskottet består av Anders Ströby (ordförande), Christian Thiel och Mikael Bluhme. Verkställande direktören är föredragande i frågor som inte berör verkställande direktörens villkor. Ersättningsutskottet har till uppgift att granska och ge styrelsen rekommendationer angående principerna för villkor och ersättning till bolagets ledande befattningshavare. Ersättningsutskottet ska årligen utvärdera verkställande direktörens arbete. Frågor som rör verkställande direktörens anställningsvillkor, ersättning och förmåner bereds av ersättningsutskottet och beslutas av styrelsen.

## Verkställande direktör och ledning

Verkställande direktören är i sin roll underordnad styrelsen och har som huvuduppgift att sköta FlexQubes löpande förvaltning och dagliga verksamheten. Av styrelsens arbetsordning och instruktionen för verkställande direktören framgår vilka frågor som FlexQubes styrelse ska fatta beslut om och vilka beslut som faller på den verkställande direktören. Verkställande direktören tar även fram rapporter och nödvändigt beslutsunderlag inför styrelsesammanträden och är föredragande av materialet vid

## Intern kontroll

Styrelsens ansvar för den interna kontrollen regleras i aktiebolagslagen och årsredovisningslagen som innehåller krav på att information om de viktigaste inslagen i FlexQubes system för intern kontroll och riskhantering i samband med den finansiella rapporteringen varje år ska ingå i bolagsstyrningsrapporten.

Styrelsen ska bland annat se till att FlexQube har god intern kontroll och formaliserade rutiner som säkerställer att fastlagda principer för finansiell rapportering och intern kontroll efterlevs samt att det finns ändamålsenliga system för uppföljning och kontroll av bolagets verksamhet och de risker som FlexQube och dess verksamhet är förknippad med.

Den interna kontrollens övergripande syfte är att i rimlig grad säkerställa att bolagets operativa strategier och mål följs upp och att ägarnas investering skyddas. Den interna kontrollen ska vidare säkerställa att den externa finansiella rapporteringen med rimlig säkerhet är tillförlitlig och upprättad i överensstämmelse med god redovisningssed, att tillämpliga lagar och förordningar följs samt att krav på noterade bolag efterlevs.

### Kontrollmiljö

Styrelsen har det övergripande ansvaret för den interna kontrollen avseende den finansiella rapporteringen. I syfte att skapa och vidmakthålla en fungerande kontrollmiljö har styrelsen antagit ett antal policyer och styrdokument som reglerar den finansiella rapporteringen.

Dessa utgörs huvudsakligen av styrelsens arbetsordning, VD-instruktion samt instruktion för finansiell rapportering. Styrelsen har också antagit en särskild attestordning samt en risk- och finanspolicy.

Styrelsen följer löpande bolagets finansiella ställning och effektiviteten i bolagets interna kontroll och riskhantering, samt håller sig informerad om revisionen av årsredovisningen och koncernredovisningen.

### Riskbedömning

Styrelsen får ytterligare information om riskhantering, intern kontroll och finansiell rapportering från revisorn via revisionsutskottet eller vid styrelsemöten med revisorn närvarande.

### Kontrollaktiviteter

Kontrollaktiviteter begränsar identifierade risker och säkerställer korrekt och tillförlitlig finansiell rapportering. Styrelsen ansvarar för den interna kontrollen och uppföljning av bolagsledningen. Detta sker genom både interna och externa kontrollaktiviteter samt genom granskning och uppföljning av bolagets styrdokument. En viktig del i FlexQubes kontrollaktiviteter är standardiserade rapporteringsrutiner samt tydliga och dokumenterade arbetsordningar och ansvarsfördelningar.

### Styrning och uppföljning

Löpande uppföljning av resultatutfall sker på flera nivåer i koncernen, såväl på bolagsnivå som på koncernnivå. Uppföljning sker mot budget och prognoser. Avrapportering sker till verkställande direktör och styrelse. Enligt styrelsens instruktion till revisionsutskottet ska utskottet årligen ha minst ett möte där bolagets revisorer deltar utan närvaro av anställda i FlexQube. Revisorerna ska rapportera om revisionens inriktning och omfattning samt om revisorernas iakttagelser gjorda vid revisionen och av den interna kontrollen. Revisionsutskottets sammanträden protokollförs och protokoll tillhandahålls till styrelsen.

### Behov av internrevision

Effektiviteten vid internrevision är till stor del beroende av bolagets organisationsstruktur och organisationens storlek. FlexQube har en relativt liten organisation där såväl finans-, ekonomi- som annan administration sköts från bolagets kontor i Göteborg. Uppföljning av resultat och balans görs månadsvis av såväl de olika funktionerna inom bolaget som av bolagsledningen samt styrelse. Sammantaget medför detta att det

## Årlig verksamhetsplan

| Period | Styrelsen | Revisionsutskott | Ersättningsutskott |
|--------|-----------|------------------|-------------------|
| **DECEMBER** | • Budget och affärsmål<br>• Bolagsstyrning<br>• Uppföljning och utvärdering av styrelsens arbete | | • Utvärdering av VD:s arbete och ledningsgrupp samt planering för framtida behov |
| **SEPTEMBER-NOVEMBER** | • Kvartalsrapport tredje kvartalet<br>• Strategisk utveckling, utvärdering, strategisk inriktning och mål | • Revisionsplanering bokslut<br>• Utvärdering av internkontroll och riskhantering | |
| **AUGUSTI** | • Kvartalsrapport andra kvartalet | | |

## Bolagsstyrning (fortsättning)

### Revisorns opartiskhet och självständighet
Styrelsen får även rapporter från FlexQubes revisor. Styrelsen anser inte att det är motiverat att ha en särskild enhet för internrevision.

### Styrelsesammanträden
Förutom verkställande direktören har FlexQube sju ledande befattningshavare, se sidorna 128-131.

### Övriga upplysningar avseende styrelse och ledning

Styrelsens ledamöter Mikael Bluhme och Anders Ströby är oberoende i förhållande till FlexQube, bolagets ledning samt dess huvudägare. Anders Fogelberg, verkställande direktör, Christian Thiel, styrelseordförande, samt Per Augustsson, CTO, innehar mer än 10 procent av kapital och röster i FlexQube AB och anses därmed vara beroende i förhållandet till större aktieägare och bolaget.

Ingen av styrelseledamöterna eller medlemmarna av koncernledningen har någon familjerelation med någon annan styrelseledamot eller medlem av koncernledningen inom FlexQube. Ingen av styrelseledamöterna eller medlemmarna av koncernledningen har under de senaste fem åren:

- dömts i bedrägerirelaterade mål,
- varit ställföreträdare för något företag som försatts i konkurs eller likvidation, med undantag för vad som framgår av sammanställningen av pågående och tidigare uppdrag för respektive person under rubriken "Styrelse" på sidan 125, eller
- av domstol förbjudits att agera som styrelseledamot eller ledande befattningshavare eller på annat sätt idka näringsverksamhet

Ingen av styrelseledamöterna eller medlemmarna av koncernledningen har varit utsatt för officiella anklagelser eller sanktioner av i lag eller förordning bemyndigade myndigheter.

Det föreligger inte heller några intressekonflikter, genom vilka styrelseledamöters eller medlemmarna av koncernledningens privata intressen skulle stå i strid med bolagets intressen.

### Närståendetransaktioner

Inget bolag i koncernen har ingått avtal som ger styrelseledamöter eller medlemmar i ledningsgruppen rätt till förmåner efter det att respektive uppdrag har avslutats. Personer i ledningsgruppen har rätt till lön under uppsägningstiden.

## Revisors yttrande om bolagsstyrningsrapporten

Till bolagsstämman i FlexQube AB (publ), org.nr 556905-3944

### UPPDRAG OCH ANSVARSFÖRDELNING
Det är styrelsen som har ansvaret för bolagsstyrningsrapporten på sidorna 116-122 och för att den är upprättad i enlighet med årsredovisningslagen.

### GRANSKNINGENS INRIKTNING OCH OMFATTNING
Vår granskning har skett enligt FARs uttalande RevU 16 Revisorns granskning av bolagsstyrningsrapporten. Detta innebär att vår granskning av bolagsstyrningsrapporten har en annan inriktning och en väsentligt mindre omfattning jämfört med den inriktning och omfattning som en revision enligt International Standards on Auditing och god revisionssed i Sverige har. Vi anser att denna granskning ger oss tillräcklig grund för våra uttalanden.

### UTTALANDE
En bolagsstyrningsrapport har upprättats. Upplysningar i enlighet med 6 kap. 6 § andra stycket punkterna 2–6 årsredovisningslagen samt 7 kap. 31 § andra stycket samma lag är förenliga med årsredovisningens och koncernredovisningens övriga delar samt är i överensstämmelse med årsredovisningslagen.

Göteborg den dag som framgår av vår elektroniska signatur.

PricewaterhouseCoopers AB  
Johan Palmgren  
Auktoriserad revisor

### Information och kommunikation

Bolaget har informations- och kommunikationsvägar som syftar till att främja riktigheten av den finansiella rapporteringen och möjliggöra rapportering och återkoppling från verksamheten till styrelse och ledning, exempelvis genom att styrande dokument i form av interna policyer, riktlinjer och instruktioner avseende den ekonomiska rapporteringen gjorts tillgängliga och är kända för berörda medarbetare.

Bolaget har även upprättat policy och instruktioner i syfte att informera anställda och andra berörda inom FlexQube om de lagar som är tillämpliga avseende bolagets informationsgivning och de särskilda krav på personer som är aktiva i ett börsnoterat bolag i situationer som exempelvis hanteringen av insiderinformation. I samband med detta har FlexQube etablerat rutiner för hantering och begränsning av spridningen av information som ännu inte offentliggjorts, så kallad uppskjutet offentliggörande.

Styrelsen har antagit en kommunikationspolicy och insiderpolicy som reglerar koncernens informationsgivning.

### Uppföljning

Efterlevande och effektiviteten i de interna kontrollerna följs upp löpande. Verkställande direktören ser till att styrelsen löpande erhåller rapportering om utvecklingen av bolagets verksamhet, däribland utvecklingen av FlexQubes resultat och ställning samt information om viktiga händelser.

### Ersättningar till styrelse och ledande befattningshavare

#### Ersättning till styrelsen
Årsstämman, den 12 maj 2022, beslutade att arvode till styrelsen skulle utgå med totalt 550 TSEK, varav 250 TSEK till styrelsens ordförande och 150 TSEK till varje ledamot som inte är anställd i bolaget.

#### Ersättning till ledande befattningshavare
Den totala kostnadsförda bruttoersättningen till verkställande direktören och koncernledningen, inklusive grundlön, pensionsbetalning samt bil och sjukförsäkringsförmån uppgick under 2022 till 9,9 MSEK varav 2,0 MSEK utgjorde ersättning till verkställande direktören.

### Revision

FlexQubes revisorer granskar årsbokslut och årsredovisning samt bolagets löpande verksamhet och rutiner för att sedan uttala sig om redovisningen samt styrelsens och verkställande direktörens förvaltning. Revisorerna ska efter varje räkenskapsår lämna en revisionsberättelse till bolagsstämman.

Bolagets revisorer rapporterar till styrelsen varje år personligen sina iakttagelser från granskningen och sina bedömningar av bolagets interna kontroll.

Vid årsstämman den 12 maj 2022 valdes Pricewaterhousecoopers AB till revisionsbolag, med Johan Palmgren som huvudansvarig revisor, samt att ersättning till revisorn ska utgå enligt godkänd räkning. För information om ersättning till revisor, se not 5.

## Styrelse

Enligt FlexQubes bolagsordning ska styrelsen bestå av lägst tre och högst sex ledamöter utan suppleanter. FlexQubes styrelse består för närvarande av fyra ledamöter, inklusive styrelseordförande, vilka tillsatts intill slutet av årsstämman 2023. Nedan förtecknas dels uppgifter om respektive styrelseledamots oberoende i förhållande till bolaget och dess ledning samt bolagets större aktieägare, dels en presentation över respektive styrelseledamot med uppgift om bl.a. namn, födelseår, år för inval i styrelsen, utbildning och erfarenhet, pågående och avslutade uppdrag sedan fem år och aktieinnehav i bolaget. Med aktieinnehav i bolaget omfattas eget, direkt och indirekt innehav samt närståendes innehav. Andra eller tidigare uppdrag inom FlexQube-koncernen anges inte.

### Oberoende i förhållande till:

| Namn | Befattning | Ledamot sedan | Bolaget och dess ledning | Större aktieägare |
|------|------------|---------------|-------------------------|-------------------|
| Christian Thiel | Styrelseordförande | 2012 | Nej | Nej |
| Per Augustsson | Styrelseledamot | 2012 | Nej | Nej |
| Anders Ströby | Styrelseledamot | 2017 | Ja | Ja |
| Mikael Bluhme | Styrelseledamot | 2020 | Ja | Ja |

### Christian Thiel
**Medgrundare och styrelseledamot sedan 2012, född 1981.**

**Utbildning**  
Jur.kand Handelshögskolan vid Göteborgs universitet och kandidatexamen i industriell och finansiell ekonomi från Handelshögskolan vid Göteborgs universitet.

**Pågående uppdrag**  
Styrelseordförande i Svensk Tillgångsutveckling AB, Feldthusen Invest AB, Litigium Capital Investment AB och F&T Equity AB.

**Bakgrund**  
Christian grundade FlexQube AB (publ) 2012 (dotterbolaget FlexQube Europe AB grundades 2010) tillsammans med Per Augustsson och Anders Fogelberg och har tidigare bland annat arbetat som VD för Svensk Tillgångsutveckling, investeringschef på F&T Equity AB och jurist på Mannheimer Swartling Advokatbyrå med inriktning mot företagsöverlåtelser. Christian har flera års erfarenhet av att starta och driva bolag inom en rad olika branscher.

**Innehav i bolaget**  
Christian Thiel innehar, genom bolag, 1 945 000 aktier i bolaget.

### Per Augustsson
**Medgrundare och styrelseledamot sedan 2012, född 1981.**

**Utbildning**  
Civilingenjör Maskinteknik vid Chalmers Tekniska Högskola.

**Pågående uppdrag**  
Styrelseledamot i AuguTech AB och styrelsesuppleant i Mostly AB.

**Bakgrund**  
Per grundade FlexQube AB (publ) 2012 (dotterbolaget FlexQube Europe AB grundades 2010) tillsammans med Christian Thiel och Anders Fogelberg och har tidigare arbetat bland annat på Volvo CE som projektledare med fokus på produktutveckling och investeringar.

**Innehav i bolaget**  
Per Augustsson innehar, genom bolag, 1 458 443 aktier i bolaget samt 7000 teckningsoptioner.

### Anders Ströby
**Styrelseledamot sedan 2017, född 1953.**

**Utbildning**  
Civilingenjör Maskinteknik, Kungliga Tekniska Högskolan (KTH), ekonomistudier vid Stockholms Universitet och studier i Business Management vid Göteborgs Management Institut.

**Pågående uppdrag**  
Styrelseledamot i Bravik Mgm AB samt Senior Advisor i ISEA Sweden KB.

**Bakgrund**  
Anders har lång erfarenhet från en rad olika uppdrag och roller inom Husqvarna-koncernen, innefattande bland annat arbete i koncernledningen och VD för divisionen Construction (uppdrag som styrelseledamot och ordförande i dotterbolag till Husqvarna AB), uppdrag som ledande befattningshavare i Electrolux Outdoor Products Ltd, styrelseordförande i Hebei Jikai Industrial co, Shijuazhuang, (Kina). Dessförinnan arbetade Anders bland annat som VD för Jonsered AB.

**Innehav i bolaget**  
Anders Ströby innehar, genom bolag och närstående, 70 000 aktier i bolaget och 22 000 teckningsoptioner.

### Mikael Bluhme
**Styrelseledamot sedan 2020, född 1966.**

**Utbildning**  
Studerat ekonomi vid IHM Business School och Executive Foundation Lund.

**Pågående uppdrag**  
Styrelseledamot i Villman & Co AB, Öresund Heavy Industrins, Nordic Kitchen Group samt CLAMO AB.

**Bakgrund**  
Mikael Bluhme är partner i RoosGruppen AB sedan 2013 och har under de senaste 30 åren arbetat, med och för, entreprenörer. Mikael har lång erfarenhet ifrån en olika entreprenörsdrivna bolag där hans fokus har varit på affärsutveckling, försäljning, marknadsföring, förändrings- och utvecklingsarbete, nationellt såväl som internationellt.

**Innehav i bolaget**  
RoosGruppen AB äger 1 231 529 aktier i bolaget och Mikael Bluhme innehar personligen 31 300 aktier och 22 000 teckningsoptioner.

## Ledande befattningshavare

FlexQubes ledande befattningshavare består för närvarande av åtta personer; Anders Fogelberg, Heidi Bader, Mikael Lindbäck, Per Augustsson, Mikaela Benjaminsson, Luke Goodwin, Todd Nethery och Michael Gore. Nedan finns en förteckning över de ledande befattningshavarnas befattningar och anställningstid, födelseår, bakgrund, aktieinnehav i bolaget samt pågående uppdrag. Andra eller tidigare uppdrag i FlexQube-koncernen anges inte. Med aktieinnehav i bolaget omfattas eget, direkt och indirekt innehav samt närståendes innehav.

### Anders Fogelberg
**Medgrundare och verkställande direktör i koncernen sedan 2013, född 1981.**

**Utbildning**  
Civilingenjör Maskinteknik vid Chalmers tekniska högskola och kandidatexamen i industriell och finansiell ekonomi från Handelshögskolan vid Göteborgs universitet.

**Pågående uppdrag**  
Styrelseledamot i Birdmountain Invest AB och styrelseledamot i FlexQubes dotterbolag.

**Bakgrund**  
Anders grundade FlexQube AB (publ) 2012 (dotterbolaget FlexQube Europe AB grundades 2010) tillsammans med Christian Thiel och Per Augustsson och har tidigare bland annat varit VD för F&T Equity AB och arbetat som managementkonsult på Accenture med inriktning mot Supply Chain Management. Anders har flera års erfarenhet från att driva egna bolag.

**Innehav i bolaget**  
Anders Fogelberg innehar 946 483 aktier i bolaget samt 17 000 teckningsopotioner.

### Per Augustsson
**Medgrundare och Chief Technology Officer sedan 2013, född 1981.**

**Utbildning**  
Civilingenjör Maskinteknik vid Chalmers Tekniska Högskola.

**Pågående uppdrag**  
Styrelseledamot i AuguTech AB och FlexQubes dotterbolag samt styrelsesuppleant i Mostly AB.

**Bakgrund**  
Per grundade FlexQube AB (publ) 2012 (dotterbolaget FlexQube Europe AB grundades 2010) tillsammans med Christian Thiel och Anders Fogelberg och har tidigare arbetat bland annat på Volvo CE som projektledare med fokus på produktutveckling och investeringar.

**Innehav i bolaget**  
Per Augustsson innehar 1 458 443 aktier i bolaget samt 7 000 teckningsoptioner.

### Heidi Bader
**(Föräldraledig) Chief Financial Officer sedan 2021, född 1981.**

**Utbildning**  
Civilekonom från Handelshögskolan vid Göteborgs universitet, MSc in Management and BSc in Accounting.

**Bakgrund**  
Heidi har närmast arbetat på DB Schenker och tidigare på AB Volvo. Heidi har lång internationell erfarenhet inom finansiell rapportering, affärskontroll och affärsplanering. Heidi har ett datadrivet tillvägagångssätt och tillför färdigheter i att driva förändring för att katalysera företagets prestation.

**Innehav i bolaget**  
Heidi innehar 1600 aktier i bolaget samt 16 000 teckningsopotioner.

### Luke Goodwin
**Chief Marketing Officer sedan 2017, född 1988.**

**Utbildning**  
Kandidatexamen i Företagsekonomi med inriktning på Marknadsföring från USC, Australien.

**Bakgrund**  
Luke kom till FlexQube med tidigare erfarenhet inom marknadsföring av mjukvara och byggbranschen med fokus på digital marknadsföring, SEO & innehållsskapande. Han har arbetat med FlexQube i över fyra år och etablerat marknadsavdelningen.

**Innehav i bolaget**  
Luke Goodwin innehar 586 aktier i bolaget samt 15 500 aktieoptioner.

### Mikael Lindbäck
**TF Chief Financial Officer, född 1982.**

**Utbildning**  
Magisterexamenen redovisning från Handelshögskolan vid Göteborgs universitet.

**Bakgrund**  
Mikael har lång erfarenhet från finansiella positioner och har tidigare arbetat som CFO för Gigant AB och Grunda AB inom B&B.

### Todd Nethery
**Plant Manager North America sedan 2021, född 1962.**

**Utbildning**  
Kandidatexamen i industriell förvaltning från Clemson University, USA.

**Bakgrund**  
Todd har en mångsidig bakgrund inom automotiv verkstads- och driftshantering, försäljning av intäktsgenerering, produktion.

### Michael Gore
**VP Sales North America sedan 2021, född 1962.**

**Utbildning**  
Mastersexamen i företagsekonomi från Virginia Commonwealth University, USA.

**Bakgrund**  
Michael har lång erfarenhet som försäljningschef inom detaljhandel med goda kunskaper av kundservice, försäljning, ledarskap, försäljningsoperationer och kontohantering.

**Innehav i bolaget**  
Michael Gore innehar 6 000 teckningsoptioner i bolaget.

## Mikaela Benjaminsson
**HR Business Partner sedan 2022, född 1989.**

### Utbildning
Magisterexamenen i arbetvetenskap från Göteborgs Univeristet.

### Bakgrund
Mikaela började på FlexQube i September 2022. Mikaela har tidigare erfarenhet från företag som AGN, Serneke Group.

### Innehav i bolaget
Mikaela Benjaminsson innehar 7 000 teckningsoptioner i bolaget.

---

## Årsstämma och finansiell kalender

**FlexQube AB:s årsstämma hålls den 15 maj, 2023.**

### Deltagande
Rätt att delta har den aktieägare som dels är införd i den av Euroclear Sweden AB förda aktieboken den 8 maj 2023, dels anmäler avsikt att delta på årsstämman till FlexQube AB senast den 8 maj 2023. De aktieägare som låtit förvaltaregistrera sina aktier måste, för att äga rätt att delta i årsstämman, tillfälligt inregistrera aktierna i eget namn hos Euroclear Sweden AB. Sådan omregistrering bör begäras i god tid före den 8 maj 2023 hos den bank eller fondhandlare som förvaltar aktierna. Aktieägarens rätt vid årsstämman får utövas genom ombud. Företrädes juridisk person ska fullmakt vara undertecknad av firmatecknare och kopia av ett aktuellt registreringsbevis som anger firmatecknare ska bifogas.

### Anmälan
Anmälan till årsstämma kan ske per post till FlexQube AB, Neongatan 8, 431 53 Mölndal eller per e-post: ir@flexqube.com. Vid anmälan anges namn, adress, telefonnummer, person- eller organisationsnummer samt namn på eventuella biträden.

### Distributionspolicy
Årsredovisningen tillhandahålls som en PDF på www.flexqubegroup.com och tryckt årsredovisning skickas till alla som så begär. Beställning skickas till ir@flexqube.com eller per post till FlexQube AB, Neongatan 8, 431 53 Mölndal.

### Finansiell kalender
- Kvartalsrapport 1, 2023: 2023/05/10
- Årsstämma 2023: 2023/05/15
- Kvartalsrapport 2, 2023: 2023/08/09
- Kvartalsrapport 3, 2023: 2023/11/08
- Kvartalsrapport 4, 2023: 2024/02/21

---

## Kontakt

**FlexQube AB (publ)**  
556905-3944  
Neongatan 8  
431 53 Mölndal, Sverige  
www.flexqubegroup.com

**Anders Fogelberg**  
CEO  
Tel +46 702-86 06 74  
anders.fogelberg@flexqube.com

**Mikael Lindbäck**  
CFO  
Tel +46 727-11 14 77  
mikael.lindback@flexqube.com

**Investor Relations**  
ir@flexqube.com  
Tel +46 761-04 10 28

**Certifierad Rådgivare**  
FNCA SWEDEN AB  
Tel +46 8-528 00 399  
info@fnca.se

---

## Övriga upplysningar avseende styrelse och ledande befattningshavare

Samtliga styrelseledamöter och ledande befattninshavare i FlexQube kan nås via adressen till FlexQubes huvudkontor, Neongtan 8, 431 53, Mölndal, Sverige.

Inga styrelseledamöter eller medlemmar av koncernledningen har några familjeband till några andra styrelseledamöter eller medlemmar av koncernledningen. Det föreligger inga intressekonflikter eller potentiella intressekonflikter mellan styrelseledamöternas och ledande befattningshavares åtaganden gentemot FlexQube och deras privata intressen och/eller andra åtaganden. Dock har flera styrelseledamöter och ledande befattningshavare finansiella intressen i FlexQube till följd av deras indirekta aktieinnehav i bolaget.

Ingen av ovan nämnda styrelseledamöter eller ledande befattningshavare har ingått avtal med något koncernbolag om förmåner efter uppdragets avslutande.

---

## Revisor

I enlighet med bolagsordningen ska FlexQube utse en eller två revisorer med högst två revisorssuppleanter eller ett registrerat revisionsbolag. Bolagets revisor utses av årsstämman. Revisorn ska granska FlexQubes årsredovisning och räkenskaper samt styrelsens och VD:s förvaltning. Bolagets revisor informeras löpande om bolagets verksamhet bland annat genom regelbundna möten med bolagsledningen, utskickat styrelsematerial samt protokoll. Revisorn kan löpande komma att lämna synpunkter och rekommendationer till bolagets styrelse och ledning. Efter varje räkenskapsår ska revisorn lämna en revisionsberättelse och en koncernrevisionsberätelse till årsstämman.

Vid årsstämma 12 maj, 2022 valdes PricewaterhouseCoopers AB som registrerat revisionsbolag med Johan Palmgren som huvudansvarig revisor. Johan Palmgren är auktoriserad revisor och är medlem i FAR. Ersättning till revisor utgår enligt godkänd löpande räkning.

---

## Källhänvisningar för fotografier:
- Corey Masser
- Zack Peeler
- Matilda Odell
- Stock Photos
- Agnes Ahlman
- Drew Peden

---

**(c) FQ IP AB 2022**

*Eftertryck får ske endast med FlexQubes skriftliga medgivande i förväg. Rapporten är ursprungligen skriven på svenska och översatt till engelska. Vid eventuella avvikelser mellan de två versionerna har den svenska versionen företräde.*